In [33]:
import numpy as np

# 1. Загрузка словаря эмбеддингов
# Файл 'russian_cleanlit_SVD_32_dict.npy' содержит словарь (key->embedding),
# сохранённый через pickle (allow_pickle=True).
data = np.load('ru_cbow_dictionary.npy', allow_pickle=True).item()

# 2. Формируем список всех ключей (слов, id и т.д.)
keys = list(data.keys())
N = len(keys)
print(f"Всего эмбеддингов: {N}")

# 3. Определяем размер выборки (1%)
percentage = 0.01
sample_size = max(1, int(N * percentage))

# 4. Выбираем случайные ключи без повторений
random_keys = np.random.choice(keys, sample_size, replace=False)
print(f"Выбрано {len(random_keys)} эмбеддингов (1%)")

# 5. Формируем новый словарь с выбранными эмбеддингами
subset_data = {k: data[k] for k in random_keys}

# 6. Сохраняем этот словарь в новый файл (также в pickle-формате),
# чтобы можно было загрузить в Julia или конвертировать в другой формат.
np.save('ru_cbow_dictionary_subset.npy', subset_data, allow_pickle=True)

print("Субсет эмбеддингов сохранён в 'ru_cbow_dictionary_subset.npy'")

Всего эмбеддингов: 72111
Выбрано 721 эмбеддингов (1%)
Субсет эмбеддингов сохранён в 'ru_cbow_dictionary_subset.npy'


In [34]:
import numpy as np

# Загружаем субсет (словарь) из предыдущего шага
subset_data = np.load('ru_cbow_dictionary_subset.npy', allow_pickle=True).item()

# Нормализуем: приводим ключи к строкам, а эмбеддинги к float64
data_normalized = {}
for k, v in subset_data.items():
    key_str = str(k)
    data_normalized[key_str] = np.array(v, dtype=np.float64)

# Сохраняем в формате .npz
np.savez('ru_cbow_dictionary_subset.npz', **data_normalized)

print("Субсет эмбеддингов теперь в файле 'russian_cleanlit_SVD_32_subset.npz'")

Субсет эмбеддингов теперь в файле 'russian_cleanlit_SVD_32_subset.npz'


In [9]:
# После создания subset_data
for k, v in subset_data.items():
    if v is None:
        print(f"Обнаружено None значение для ключа: {k}")
    if np.isnan(v).any():
        print(f"Обнаружены NaN значения для ключа: {k}")

# После создания data_normalized
for k, v in data_normalized.items():
    if np.isnan(v).any():
        print(f"Обнаружены NaN значения для ключа: {k}")

#### тут всё в разработке...

In [ ]:
конвектор на npz:

In [4]:
import numpy as np

def convert_to_npz(input_file='urmi_dictionary_photo.txt', output_file='urmi_dictionary_photo.npz'):
    # Словарь для хранения слов и эмбеддингов
    words = []
    embeddings = []
    
    # Чтение файла
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            word = parts[0]
            embedding = [float(x) for x in parts[1:]]
            
            # Добавляем слово и эмбеддинг в списки
            words.append(word)
            embeddings.append(embedding)
    
    # Преобразование списков в numpy-массивы
    words_array = np.array(words, dtype=object)  # Массив слов
    embeddings_array = np.vstack(embeddings)  # Создание двумерного массива из списка эмбеддингов
    
    # Сохранение в файл .npz
    np.savez(output_file, words=words_array, embeddings=embeddings_array)

# Вызов функции для преобразования
convert_to_npz()


In [5]:
import numpy as np
data = np.load('urmi_dictionary_photo.npz',  allow_pickle=True)
print(data.files)
print(data['words'])
print(data['embeddings'])

['words', 'embeddings']
['min' 'bit' 'pjəşə' ... 'dmadxurilli' 'çaponaja' 'prədə']
[[-0.09148435  0.12686516  0.50513256 ... -0.46129158 -0.04712095
  -0.21865861]
 [-0.20468658 -0.65338475  1.5521379  ... -1.0280646  -0.37638134
   0.64581996]
 [-2.3566349  -0.43647033  0.11017048 ... -0.13083504 -1.2178925
   0.59010524]
 ...
 [-0.00305589 -0.0762874   0.05905655 ... -0.32206282 -0.13641188
  -0.10338827]
 [ 0.00737996 -0.13167177  0.03114334 ... -0.25681385 -0.09841377
  -0.15485328]
 [ 0.03784949 -0.05858647 -0.06285505 ... -0.20090392 -0.06449439
  -0.09001213]]


In [12]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary.npz', allow_pickle=True)

# Преобразование слов в массив строк (UTF-8)
words = np.array([str(word) for word in data['words']], dtype='<U')  # Массив строк
embeddings = np.array(data['embeddings'], dtype=np.float64)  # Массив чисел с типом float64

# Сохранение нового файла .npz без объектов
np.savez('urmi_dictionary_fixed.npz', words=words, embeddings=embeddings)


In [13]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary_fixed.npz', allow_pickle=True)

# Сохранение нового файла .npz с только эмбеддингами
np.savez('urmi_dictionary_embeddings.npz', embeddings=data['embeddings'])


In [20]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary_fixed.npz', allow_pickle=True)

# Выборка 50% эмбеддингов
N = len(data['embeddings'])
sample_size = int(N * 0.7)  # 50% от общего количества

# Случайный выбор индексов для выборки
random_indices = np.random.choice(N, sample_size, replace=False)

# Формирование нового массива эмбеддингов
subset_embeddings = data['embeddings'][random_indices]

# Сохранение нового файла .npz с выборкой эмбеддингов
np.savez('urmi_dictionary_subset.npz', embeddings=subset_embeddings)

print(f"Сохранено {sample_size} эмбеддингов (50%) в 'urmi_dictionary_subset.npz'")


Сохранено 4132 эмбеддингов (50%) в 'urmi_dictionary_subset.npz'


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from tqdm.auto import tqdm
import pandas as pd

In [21]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary_fixed.npz', allow_pickle=True)

# Преобразование строк в массив байтов
words_bytes = np.array([w.encode('utf-8') for w in data['words']])

# Сохранение нового файла .npz с преобразованными данными
np.savez('urmi_dictionary_fixed_bytes.npz', words=words_bytes, embeddings=data['embeddings'])


In [22]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary_fixed.npz', allow_pickle=True)

# Преобразование строк в массив байтов
words_bytes = np.array([w.encode('utf-8') for w in data['words']])

# Сохранение нового файла .npz с преобразованными данными
np.savez('urmi_dictionary_fixed_bytes.npz', words=words_bytes, embeddings=data['embeddings'])



In [23]:
import numpy as np

# Загрузка исходного файла .npz
data = np.load('urmi_dictionary_fixed.npz', allow_pickle=True)

# Сохранение нового файла .npz с только эмбеддингами
np.savez('urmi_dictionary_embeddings.npz', embeddings=data['embeddings'])

In [18]:
import numpy as np

# Укажите путь к файлу
file_path = '/home/kuklelik/Jupyter/ru_word_holes.npy'

# Загрузка файла
data = np.load(file_path, allow_pickle=True)

# Отображение содержимого
print(data)

[[707 667]
 [707 231]
 [534 231]
 ...
 [153  65]
 [146  65]
 [146 109]]


In [7]:
import numpy as np

data = np.load('ru_cbow_dictionary.npy', allow_pickle=True).item()

# Проверка
first_key = next(iter(data)) # Получаем первый ключ словаря
print(f"Тип ключа: {type(first_key)}")
print(f"Тип значения (эмбеддинга): {type(data[first_key])}")
print(f"Размерность эмбеддинга: {data[first_key].shape}")
print(f"Первые 10 значений эмбеддинга: {data[first_key][:10]}")

Тип ключа: <class 'numpy.str_'>
Тип значения (эмбеддинга): <class 'numpy.ndarray'>
Размерность эмбеддинга: (100,)
Первые 10 значений эмбеддинга: [-3.6233642  -1.3633608   1.5723407  -0.77882737 -0.28940833  1.1254197
  0.17674242 -0.5417739   1.7357608  -1.0002639 ]


In [10]:
np.save('ru_cbow_dictionary_subset_test.npy', subset_data, allow_pickle=True)
subset_data_loaded = np.load('ru_cbow_dictionary_subset_test.npy', allow_pickle=True).item()

# Проверьте тип и значения subset_data_loaded
first_key = next(iter(subset_data_loaded))
print(f"Тип ключа после повторной загрузки: {type(first_key)}")
print(f"Тип значения после повторной загрузки: {type(subset_data_loaded[first_key])}")
print(f"Размерность эмбеддинга после повторной загрузки: {subset_data_loaded[first_key].shape}")
print(f"Первые 10 значений эмбеддинга после повторной загрузки: {subset_data_loaded[first_key][:10]}")

Тип ключа после повторной загрузки: <class 'numpy.str_'>
Тип значения после повторной загрузки: <class 'numpy.ndarray'>
Размерность эмбеддинга после повторной загрузки: (100,)
Первые 10 значений эмбеддинга после повторной загрузки: [-3.6233642  -1.3633608   1.5723407  -0.77882737 -0.28940833  1.1254197
  0.17674242 -0.5417739   1.7357608  -1.0002639 ]


In [17]:
import numpy as np

# Создаём небольшой словарь для тестирования
test_data = {'word1': np.array([1.0, 2.0, 3.0]), 'word2': np.array([4.0, 5.0, 6.0])}

# Сохраняем в .npz
np.savez('test_subset.npz', **test_data)

# Загружаем обратно
loaded_data = np.load('test_subset.npz')

# Выводим ключи и значения
print("Ключи в загруженном файле:", loaded_data.files)
for key in loaded_data.files:
    print(f"Значение для ключа '{key}': {loaded_data[key]}")
    print(f"Тип значения для ключа '{key}': {type(loaded_data[key])}")

Ключи в загруженном файле: ['word1', 'word2']
Значение для ключа 'word1': [1. 2. 3.]
Тип значения для ключа 'word1': <class 'numpy.ndarray'>
Значение для ключа 'word2': [4. 5. 6.]
Тип значения для ключа 'word2': <class 'numpy.ndarray'>


In [24]:
import numpy as np

# Пути к файлам
full_dictionary_path = '/home/kuklelik/Jupyter/ru_cbow_dictionary.npy'
subset_dictionary_path = '/home/kuklelik/Jupyter/ru_cbow_dictionary_subset.npz'
original_cycles_path = '/home/kuklelik/Jupyter/ru_word_holes.npy'
new_cycles_path = '/home/kuklelik/Jupyter/ru_word_holes_subset.npy'

# 1. Загрузка исходного словаря
full_dictionary = np.load(full_dictionary_path, allow_pickle=True).item()
full_word_to_index = {word: index for word, index in full_dictionary.items()} # Словарь слово -> индекс

# Вывод нескольких элементов словаря
count = 0
print("\nПервые несколько элементов full_word_to_index:")
for word, index in full_word_to_index.items():
    print(f"Слово: {word}, Тип индекса: {type(index)}, Значение индекса: {index}")
    count += 1
    if count > 5:
        break

# 2. Загрузка подмножества словаря
subset_vars = np.load(subset_dictionary_path)
subset_word_to_index = {word: i for i, word in enumerate(subset_vars.files)} # Словарь слово -> индекс в подмножестве

# 3. Загрузка исходных циклов
original_cycles = np.load(original_cycles_path, allow_pickle=True)
original_cycles = original_cycles.astype(np.int64) # Ensure correct type

# 4. Преобразование индексов циклов
new_cycles = []
for cycle in original_cycles:
    new_cycle = []
    for original_index in cycle: # Цикл по отдельным индексам
        # Отладочные выводы
        print(f"\nТип original_index: {type(original_index)}")
        print(f"Значение original_index: {original_index}")

        # Находим слово по исходному индексу
        # Используем .item() чтобы извлечь скалярное значение из массива numpy
        word = next((word for word, index in full_word_to_index.items() if index == original_index.item()), None)

        if word is None:
            print(f"Предупреждение: Не найдено слово с индексом {original_index} в полном словаре.")
            continue # Пропускаем это слово

        # Находим индекс этого слова в подмножестве
        if word in subset_word_to_index:
            new_index = subset_word_to_index[word]
            new_cycle.append(new_index + 1) # +1, так как Julia индексирует с 1
        else:
            print(f"Предупреждение: Слово '{word}' отсутствует в подмножестве.")
            continue  # Пропускаем это слово

    if len(new_cycle) > 1: # Сохраняем только циклы, содержащие как минимум 2 слова из подмножества
        new_cycles.append(new_cycle)

# 5. Сохранение новых циклов
new_cycles_array = np.array(new_cycles, dtype=object) # Сохраняем как массив объектов, тк циклы могут быть разной длины
np.save(new_cycles_path, new_cycles_array, allow_pickle=True)

print(f"Преобразованные циклы сохранены в {new_cycles_path}")


Первые несколько элементов full_word_to_index:
Слово: аарон, Тип индекса: <class 'numpy.ndarray'>, Значение индекса: [ 3.91300768e-03 -1.37788849e-02  4.02071550e-02  2.47548819e-02
 -5.98336048e-02 -6.84712604e-02 -1.96311697e-02  5.20185195e-02
  4.18518810e-03 -6.66666031e-02 -1.44358573e-03 -2.96986904e-02
 -4.23146561e-02  2.60344311e-03 -1.81155093e-02 -1.03004545e-01
 -2.26222891e-02 -4.06192988e-02 -4.47104685e-02  1.07545974e-02
  1.73088908e-02 -7.34853521e-02  9.41435248e-03 -2.04521809e-02
 -7.04786777e-02 -3.33687924e-02  2.60091182e-02  3.94541733e-02
 -7.25762397e-02  1.62794143e-02  1.51762823e-02 -2.38894541e-02
 -4.76410091e-02 -6.83002099e-02 -2.58859769e-02 -1.77979413e-02
  6.31137565e-02 -5.16565032e-02 -7.99258500e-02  2.87138131e-02
  7.10096257e-03 -6.06296025e-02 -5.94107397e-02 -4.20297310e-02
 -5.10234013e-02 -1.48509741e-02 -5.92829660e-02 -5.17091490e-02
 -2.70023067e-02  6.65122643e-02  3.90822701e-02  1.96503382e-02
  7.13727176e-02 -2.54588202e-02 -4.3

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [22]:
import numpy as np

original_cycles_path = '/home/kuklelik/Jupyter/ru_word_holes.npy'
original_cycles = np.load(original_cycles_path, allow_pickle=True)

print(f"Тип данных original_cycles: {original_cycles.dtype}")
print(f"Пример цикла: {original_cycles[0]}")  # Выведите первый цикл


Тип данных original_cycles: int64
Пример цикла: [707 667]


In [27]:
import numpy as np

# Загружаем исходный файл
data = np.load("ru_cbow_dictionary_subset.npz", allow_pickle=True)

# Извлекаем словарь из одного объекта под ключом 'data'
real_data = data["data"].item()
words = list(real_data.keys())
vectors = np.array([real_data[w] for w in words])

# Сохраняем слова в текстовый файл (одна строка — одно слово)
with open("ru_words.txt", "w", encoding="utf-8") as f:
    for w in words:
        f.write(w + "\n")

# Сохраняем вектора как npy
np.save("ru_vectors.npy", vectors)

print(f"Сохранено {len(words)} слов и векторов.")



Сохранено 721 слов и векторов.


In [26]:
import numpy as np

data = np.load("ru_cbow_dictionary_subset.npz", allow_pickle=True)

print("Ключи в файле:")
print(data.files)
print("Тип содержимого:", type(data[data.files[0]]))

# Если только один ключ — это подозрительно
if len(data.files) == 1:
    val = data[data.files[0]]
    print("Тип значения под этим ключом:", type(val))
    
    # Если это словарь — раскладываем
    if isinstance(val.item(), dict):
        print("Файл содержит словарь внутри одного объекта. Распаковываем...")
        real_data = val.item()
        words = list(real_data.keys())
        vectors = np.array([real_data[w] for w in words])

        np.save("ru_words.npy", np.array(words))
        np.save("ru_vectors.npy", vectors)
        print(f"Сохранено {len(words)} слов и векторов.")
    else:
        print("Файл не содержит словарь. Нужен другой способ.")
else:
    # Обычный путь
    words = list(data.keys())
    vectors = np


Ключи в файле:
['data']
Тип содержимого: <class 'numpy.ndarray'>
Тип значения под этим ключом: <class 'numpy.ndarray'>
Файл содержит словарь внутри одного объекта. Распаковываем...
Сохранено 721 слов и векторов.


In [28]:
import numpy as np

# Загружаем файл
data = np.load("ru_cbow_dictionary_subset.npz")

# Печатаем содержимое
print(data.files)

# Сохраняем данные в текстовый файл
np.savetxt("ru_cbow_dictionary_subset.txt", data['arr_0'])


['data']


KeyError: 'arr_0 is not a file in the archive'

--------------------------------------------

### **Препроцессинг фотковых текстов**

#### **1. Начальная обработка**

In [1]:
import re

input_file = "recognized_all_gpt.txt"
output_file = "recognized_all_gpt_new.txt"

with open(input_file, "r", encoding="utf-8") as f:
    text = f.read()

text = re.sub(r'-\s*\n\s*', '', text)
text = re.sub(r'\n+', ' ', text)
text = re.sub(r'\s+', ' ', text).strip()
sentences = re.split(r'(?<=[.!?])(?:\s+|(?=["“”\'»)])|(?=\Z))', text)
sentences = [s.strip() for s in sentences if s.strip()]

with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(sentences))

print(f"Количество предложений: {len(sentences)}")

Количество предложений: 3662


In [10]:
import re

def clean_text(text):
    """
    Очищает текст от всех знаков препинания, кроме точки, и удаляет цифры.
    Каждое исходное предложение остается на новой строке.
    """
    lines = text.splitlines()  # Разделяем текст на строки (предложения)
    cleaned_lines = []

    for line in lines:

        line = re.sub(r"[!\"#$%&'()*+,-\/:;<=>?@\[\\\]^_`{|}~–«»…]+", "", line)
        line = re.sub(r"\d+", "", line)

        # удаляем лишние пробелы
        line = re.sub(r" +", " ", line)
        line = line.strip()


        if line:
            if not line.endswith('.'):
                 if line:
                      line += '.'
            cleaned_lines.append(line)


    return '\n'.join(cleaned_lines)

# Имена файлов
input_filename = 'recognized_all_gpt_new.txt'
output_filename = 'not_punct.txt'

try:

    with open(input_filename, 'r', encoding='utf-8') as infile:
        original_text = infile.read()


    cleaned_text = clean_text(original_text)

    with open(output_filename, 'w', encoding='utf-8') as outfile:
        outfile.write(cleaned_text)

    print(f"Файл '{input_filename}' успешно обработан.")
    print(f"Результат сохранен в файл '{output_filename}'.")

except FileNotFoundError:
    print(f"Ошибка: Файл '{input_filename}' не найден.")
    print("Пожалуйста, убедитесь, что файл находится в той же папке, что и скрипт, или укажите правильный путь.")

except Exception as e:
    print(f"Произошла ошибка при обработке файла: {e}")



Файл 'recognized_all_gpt_new.txt' успешно обработан.
Результат сохранен в файл 'not_punct.txt'.


In [11]:
import re

def clean_text_revised(text):
    """
    Очищение текста:
    1. Удаление последовательности 'd''' и 'l'''.
    2. Только буквы (из любого алфавита) и пробелы.
    3. Нет цифр и пунктуации.
    4. Нормальные пробелы
    Каждая исходная строка остается на новой строке в результате.
    """
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        processed_line = line.replace("d'", "")
        processed_line = processed_line.replace("l'", "")

        # Оставить толкьо буквы и пробелы
        filtered_chars = [char for char in processed_line if char.isalpha() or char.isspace()]
        filtered_line = "".join(filtered_chars)
        # всё с одним пробелом 
        final_line = re.sub(r'\s+', ' ', filtered_line)
        final_line = final_line.strip()

        # Добавляем непустые строки
        if final_line:
            cleaned_lines.append(final_line)

    return '\n'.join(cleaned_lines)

input_filename = 'recognized_all_gpt_new.txt'
output_filename = 'not_punct.txt'

try:
    with open(input_filename, 'r', encoding='utf-8') as infile:
        original_text = infile.read()

    cleaned_text = clean_text_revised(original_text)

    with open(output_filename, 'w', encoding='utf-8') as outfile:
        outfile.write(cleaned_text)

    print(f"Файл '{input_filename}' успешно обработан.")
    print(f"Текст после удаления 'd\'', 'l\'', цифр и пунктуации сохранен в файл '{output_filename}'.")

except FileNotFoundError:
    print(f"Ошибка: Файл '{input_filename}' не найден.")
    print("Что-то не так с путём")
except Exception as e:
    print(f"Произошла ошибка при обработке файла: {e}")


Файл 'recognized_all_gpt_new.txt' успешно обработан.
Текст после удаления 'd'', 'l'', цифр и пунктуации сохранен в файл 'not_punct.txt'.


#### **2. Сохранение именованных сущностей (топонимы) в отдельный файл**

In [6]:
import re

def find_entities(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    entities = set()
    sentences = text.split('\n')
    
    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence)
        for i, word in enumerate(words):
            #считаем именованной сущностью слово, если оно начинается с заглавной латинской буквы
            if len(word) > 1 and word[0].isupper() and word.isascii():
                # фильтр по длине
                if len(word) > 2:
                    entities.add(word)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sorted(entities)))

if __name__ == "__main__":
    input_file = 'not_punct.txt'          
    output_file = 'named_entities.txt'

    find_entities(input_file, output_file)
    print(f'Результаты в {output_file}')


Результаты записаны в named_entities.txt


#### **3. Лемматизация файла с помощью парсера urmi-parser**

In [12]:
import os
import re
from uniparser_urmi import UrmiAnalyzer
import logging

logging.basicConfig(filename='error.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s', encoding='utf-8')

def tokenize_and_lemmatize(input_file, output_file):
    """
    Токенизирует и лемматизирует предложения из файла.
    Если лемма не найдена для слова, начинающегося на "в", удаляет "в" и пробует ещё раз.
    """
    try:
        a = UrmiAnalyzer(mode='nodiacritics')
    except Exception as e:
        logging.error(f"Не удалось инициализировать UrmiAnalyzer: {e}")
        print(f"Критическая ошибка: Не удалось инициализировать UrmiAnalyzer")
        return
        
    sentences = []
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                line = line.strip()
                if line:
                    sentences.append(line)
    except FileNotFoundError:
        logging.error(f"Входной файл не найден: {input_file}")
        print(f"Ошибка: Входной файл '{input_file}' не найден.")
        return 
    except Exception as e:
        logging.error(f"Ошибка при чтении файла '{input_file}': {e}")
        print(f"Ошибка при чтении входного файла в error.log.")
        return

    results = []
    for sentence in sentences:
        try:
            lemmatized_sentence = []
            words = sentence.split() 
            for word in words:
                try:
                    analyses = a.analyze_words(word)
                    lemmatized_word_analyses = [] # анализы для конкретного слова
                    lemma_found = False

                    for ana in analyses:
                        if hasattr(ana, 'lemma') and ana.lemma: # на наличие леммы
                            lemmatized_word_analyses.append({
                                "token": ana.wf if hasattr(ana, 'wf') else word, # используем исходное слово, если wf нет
                                "lemma": ana.lemma,
                                "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                            })
                            lemma_found = True
                            break # берём первый непустой анализ

                    # попытка с удалением 'в'
                    if not lemma_found and word.lower().startswith('в') and len(word) > 1:
                        modified_word = word[1:]
                        analyses_mod = a.analyze_words(modified_word)
                        for ana in analyses_mod:
                             if hasattr(ana, 'lemma') and ana.lemma:
                                lemmatized_word_analyses.append({
                                    "token": ana.wf if hasattr(ana, 'wf') else modified_word,
                                    "lemma": ana.lemma,
                                    "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                                })
                                lemma_found = True
                                break

                    # если лемма так и не найдена после всех попыток
                    if not lemma_found:
                        lemmatized_word_analyses.append({
                                    "token": word, # исходное слово как токен
                                    "lemma": "",
                                    "gramm": "", 
                                })

                    lemmatized_sentence.append({
                        "word": word, # исходное слово из предложения
                        "analyses": lemmatized_word_analyses, # список анализов (один или пустая лемма)
                    })

                except Exception as e_word:
                    logging.error(f"Ошибка при обработке слова '{word}' в предложении '{sentence}': {e_word}")
                    # Добавка слова без анализа в случае ошибки
                    lemmatized_sentence.append({
                        "word": word,
                        "analyses": [{ "token": word, "lemma": "ОШИБКА", "gramm": str(e_word) }]
                    })

            results.append({
                "sentence": sentence,
                "lemmatized_words": lemmatized_sentence,
            })
        except Exception as e_sentence:
            logging.error(f"Критическая ошибка при обработке предложения '{sentence}': {e_sentence}")
            # Добавка предложения с пометкой об ошибке
            results.append({
                "sentence": sentence,
                "lemmatized_words": [{"word": "ОШИБКА ПРЕДЛОЖЕНИЯ", "analyses": []}],
            })

    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
           for item in results:
                outfile.write(f"Предложение: {item['sentence']}\n")
                # Проверяем, есть ли слова для вывода
                if item.get("lemmatized_words"):
                    for word_info in item["lemmatized_words"]:
                        outfile.write(f"  Слово: {word_info['word']}\n")
                        # Проверяем, есть ли анализы для слова
                        if word_info.get("analyses"):
                            for analysis in word_info["analyses"]:
                                outfile.write(f"    Токен: {analysis.get('token', '')}\n") 
                                outfile.write(f"      Лемма: {analysis.get('lemma', '')}\n")
                                outfile.write(f"      Грамматика: {analysis.get('gramm', '')}\n")
                        else:
                            outfile.write("    (Нет анализа)\n")
                else:
                     outfile.write("  (Ошибка обработки предложения)\n")
                outfile.write("\n") # Пустая строка между предложениями
    except Exception as e:
        logging.error(f"Ошибка при записи в файл '{output_file}': {e}")
        print(f"Ошибка при записи результатов в файл -  в error.log.")

# --- Основная часть скрипта ---
if __name__ == "__main__":
    # Указываем нужные имена файлов
    input_file = 'not_punct.txt'  # Ваш очищенный файл
    output_file = 'lemmatized_not_punct_all.txt' # Имя для файла с результатами

    print(f"Запуск токенизации и лемматизации для файла '{input_file}'...")
    tokenize_and_lemmatize(input_file, output_file)
    print(f"Обработка завершена. Результаты в '{output_file}'")



Запуск токенизации и лемматизации для файла 'not_punct.txt'...
Обработка завершена. Результаты сохранены в 'lemmatized_not_punct_all.txt'. Проверьте также файл 'error.log' на наличие ошибок.


In [ ]:
from uniparser_urmi import UrmiAnalyzer

def tokenize_and_lemmatize(input_file, output_file):
    """
    Токенизирует и лемматизирует предложения из файла.
    Если лемма не найдена для слова, начинающегося на "в", удаляет "в" и пробует ещё раз.
    """
    a = UrmiAnalyzer(mode='nodiacritics')
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        sentences = [line.strip() for line in infile if line.strip()]
    
    results = []
    for sentence in sentences:
        lemmatized_sentence = []
        for word in sentence.split():
            analyses = a.analyze_words(word)
            lemmatized_word_analyses = []
            lemma_found = False
            
            for ana in analyses:
                if hasattr(ana, 'lemma') and ana.lemma:
                    lemmatized_word_analyses.append({
                        "token": ana.wf if hasattr(ana, 'wf') else word,
                        "lemma": ana.lemma,
                        "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                    })
                    lemma_found = True
                    break
            
            if not lemma_found and word.lower().startswith('в') and len(word) > 1:
                modified_word = word[1:]
                analyses_mod = a.analyze_words(modified_word)
                for ana in analyses_mod:
                    if hasattr(ana, 'lemma') and ana.lemma:
                        lemmatized_word_analyses.append({
                            "token": ana.wf if hasattr(ana, 'wf') else modified_word,
                            "lemma": ana.lemma,
                            "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                        })
                        lemma_found = True
                        break
            
            if not lemma_found:
                lemmatized_word_analyses.append({
                    "token": word,
                    "lemma": "",
                    "gramm": "", 
                })
            
            lemmatized_sentence.append({
                "word": word,
                "analyses": lemmatized_word_analyses,
            })
        
        results.append({
            "sentence": sentence,
            "lemmatized_words": lemmatized_sentence,
        })
    
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for item in results:
            outfile.write(f"Предложение: {item['sentence']}\n")
            for word_info in item["lemmatized_words"]:
                outfile.write(f"  Слово: {word_info['word']}\n")
                for analysis in word_info["analyses"]:
                    outfile.write(f"    Токен: {analysis.get('token', '')}\n") 
                    outfile.write(f"      Лемма: {analysis.get('lemma', '')}\n")
                    outfile.write(f"      Грамматика: {analysis.get('gramm', '')}\n")
            outfile.write("\n")

if __name__ == "__main__":
    input_file = 'not_punct.txt'
    output_file = 'lemmatized_not_punct_all.txt'
    
    print(f"Запуск токенизации и лемматизации для файла '{input_file}'...")
    tokenize_and_lemmatize(input_file, output_file)
    print(f"Обработка завершена. Результаты в '{output_file}'")

запись нелемматизированных слов в отдельный файл

In [14]:
def find_words_without_lemma(input_file, output_file):
    """
    Файл с not_lemma
    Находит и записывает в файл слова без леммы (не короче 4 символов) из файла output_lemmatized_all.txt.
    """

    words_without_lemma = []
    current_word = None
    lemma_found = False

    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()

            if line.startswith("Слово:"):
                if current_word and not lemma_found and len(current_word) >= 4:
                    words_without_lemma.append(current_word)
                current_word = line.split(": ", 1)[1]
                lemma_found = False
            elif line.startswith("Лемма:") and current_word:
                parts = line.split(": ", 1)
                if len(parts) > 1:
                    lemma = parts[1].strip()
                    if lemma:
                         lemma_found = True

    if current_word and not lemma_found and len(current_word) >= 4:
         words_without_lemma.append(current_word)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for word in words_without_lemma:
            outfile.write(word + '\n')


if __name__ == "__main__":
    input_file = 'lemmatized_not_punct_all.txt'
    output_file = 'not_lemma_with_photo.txt'
    find_words_without_lemma(input_file, output_file)
    print(f"Слова без леммы (не короче 4 символов) записаны в файл '{output_file}'.")

Слова без леммы (не короче 4 символов) записаны в файл 'not_lemma_with_photo.txt'.


#### **4. Лемматизация нелемматизированного остатка с fuzzywuzzy**

In [18]:
import re
from fuzzywuzzy import process
from uniparser_urmi import UrmiAnalyzer
from collections import defaultdict

def get_all_urmi_lexemes(lexemes_file):
    """
    Читает лексемы из файла словаря урми и возвращает список лексем.
    """
    lexemes = set() # set для быстрой проверки наличия
    try:
        with open(lexemes_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                if line.strip().startswith("lex: "):
                    lexeme = line.strip().replace('lex: ', '')
                    lexemes.add(lexeme)
    except FileNotFoundError:
        print(f"Ошибка: Файл лексем '{lexemes_file}' не найден.")
        return None # если ошибка
    except Exception as e:
        print(f"Ошибка при чтении файла лексем '{lexemes_file}': {e}")
        return None
    return list(lexemes) y

def extract_consonants(word):
    """
    извлекает последовательность согласных из слова.
    """
    return "".join(re.findall(r"[bcdfghjklmnpqrstvwxyz’вƶşçţqk]", word, re.IGNORECASE))

def compare_consonant_sequences(original_word, lexeme):
    """
    Сравнивает последовательности согласных ОРИГИНАЛЬНОГО слова и лексемы.
    Возвращает коэффициент похожести (0-100).
    """
    word_cons = extract_consonants(original_word)
    lexeme_cons = extract_consonants(lexeme)
    if not word_cons or not lexeme_cons:
        return 0

    total_weight = 0
    weighted_match = 0
    word_idx = 0
    lexeme_idx = 0

    while word_idx < len(word_cons) and lexeme_idx < len(lexeme_cons):
        weight = 1.0
        if word_idx < 2: weight = 3.0
        elif word_idx < 3: weight = 2.0

        total_weight += weight

        if word_cons[word_idx].lower() == lexeme_cons[lexeme_idx].lower(): # сравнение без учета регистра
            weighted_match += weight
            lexeme_idx += 1
        word_idx += 1

    return (weighted_match / total_weight) * 100 if total_weight > 0 else 0


def find_best_lemmas(word, urmi_lexemes, limit=10, threshold=60):
    """
    Находит наиболее вероятные леммы для слова, используя нечеткое сравнение и сравнение последовательности согласных.
    Пробует варианты с заменой q - k
    Возвращает список кортежей (лемма, оценка_согласных).
    """
    if not urmi_lexemes: # Если лексемы не загрузились
        return []

    word_lower = word.lower()
    word_variations = {word} # Начинаем с оригинального слова


    
    #генерируем варианты с заменой q - k
    if 'q' in word_lower:
        variant_k = ''.join(['k' if c == 'q' else 'K' if c == 'Q' else c for c in word])
        word_variations.add(variant_k)
    if 'k' in word_lower:
        variant_q = ''.join(['q' if c == 'k' else 'Q' if c == 'K' else c for c in word])
        word_variations.add(variant_q)


    
    combined_results = {} #cловарь для хранения лучших результатов {лемма: лучшаяоценкасогласных}

    # Ищем (оригинал + q/k замены)
    for variant in word_variations:
        # 1й отбор с помощью fuzzywuzzy для текущего вара ( с увелич лимита)
        preliminary_matches = process.extract(variant, urmi_lexemes, limit=limit * 2)
        if not preliminary_matches:
            continue

        # 2. сравнение согласных ОРИГИНАЛЬНОГО слова и выбор подходящих лемм
        for lexeme, fuzzy_score in preliminary_matches:
            # Сравниваем последовательность согласных ОРИГИНАЛЬНОГО слова с лексемой
            consonant_score = compare_consonant_sequences(word, lexeme)

            # Фильтр по порогу согласных и длине
            if consonant_score >= threshold and len(lexeme) >= len(word) / 3:
                #сохраняем или обновляем результат, если текущая оценка лучше
                combined_results[lexeme] = max(combined_results.get(lexeme, 0), consonant_score)

    sorted_matches = sorted(combined_results.items(), key=lambda item: item[1], reverse=True)

    # Возвращаем топ N результатов после объединения и сортировки
    return sorted_matches[:limit]


def find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60):
    """
    Находит и предлагает леммы для слов, которые не смог распознать UrmiAnalyzer и записывает результаты в файл
    """
    try:
        analyzer = UrmiAnalyzer(mode='nodiacritics')
    except Exception as e:
        print(f"Критическая ошибка: Не удалось инициализировать UrmiAnalyzer: {e}")
        return

    urmi_lexemes = get_all_urmi_lexemes(lexemes_file)
    if urmi_lexemes is None: # Проверка на лексемы
        print("Не удалось загрузить лексемы.")
        return

    urmi_lexemes_set = set(urmi_lexemes) # Используем set для быстрой проверки наличия

    try:
        with open(my_words_file, 'r', encoding='utf-8') as infile, \
             open(output_file, 'w', encoding='utf-8') as outfile:

            for line in infile:
                word = line.strip()
                if not word: continue # Пропускаем пустые строки


                try:
                    analyses = analyzer.analyze_words(word)
                    has_lemma = False
                    found_lemma_str = "" # строка для хранения найденной леммы
                    if analyses:
                        for ana in analyses:
                            if hasattr(ana, 'lemma') and ana.lemma:
                                has_lemma = True
                                found_lemma_str = ana.lemma # запомнили первую найденную лемму
                                break # достаточно одной найденной леммы

                    # Если парсер нашёл лемму, записываем
                    if has_lemma:
                         outfile.write(f"Слово: {word}, Найдена лемма (UrmiAnalyzer): {found_lemma_str}\n")
                         continue # переход к следующему слову

                    # если не нашёл
                    best_lemmas = find_best_lemmas(word, urmi_lexemes, limit, threshold)
                    if best_lemmas:
                        # вывод с оценкой согл
                        lemmas_output = ', '.join([f'{lex} ({score:.1f}%)' for lex, score in best_lemmas])
                        outfile.write(f"Слово: {word}, Предложенные леммы (по согласным): {lemmas_output}\n")
                    else:
                        outfile.write(f"Слово: {word}, Не найдено подходящих лемм\n")

                except Exception as e_word:
                    print(f"Ошибка при обработке слова '{word}': {e_word}")
                    outfile.write(f"Слово: {word}, Ошибка обработки: {e_word}\n")

    except FileNotFoundError:
        print(f"Ошибка: Файл со словами '{my_words_file}' не найден.")
    except Exception as e:
        print(f"Ошибка при работе с файлами: {e}")


if __name__ == "__main__":
    my_words_file = "not_lemma_with_photo.txt"
    lexemes_file = "lexemes.txt"                
    output_file = "prob_lemmatized_not_lemma_photo_all.txt" 

    print("Запуск поиска недостающих лемм...")
    find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60)
    print(f"Поиск завершен. Результаты сохранены в '{output_file}'.")



Запуск поиска недостающих лемм...
Поиск завершен. Результаты сохранены в 'prob_lemmatized_not_lemma_photo_all.txt'.


#### **5. Расфасовка на отлемматизированные слова и всё ещё ошибки**

In [20]:
import re
import os

def remove_duplicate_lines(input_file, output_file):
    """
    Читает файл, удаляет дубликаты строк и записывает результат в новый файл.
    """
    unique_lines = set()
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                unique_lines.add(line.strip()) # Удаляем лишние пробелы по краям перед добавлением
    except FileNotFoundError:
        print(f"Ошибка: Не найден файл для удаления дубликатов: {input_file}")
        return False # Сигнализируем об ошибке

    # Сортируем строки для более предсказуемого порядка (опционально)
    sorted_lines = sorted(list(unique_lines))

    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in sorted_lines:
                # Проверяем, что строка не пустая после strip()
                if line:
                    outfile.write(line + '\n') # Добавляем перенос строки обратно
        return True # Успешно
    except Exception as e:
        print(f"Ошибка при записи уникальных строк в {output_file}: {e}")
        return False

def count_mismatched_chars(word, lemma):
    """
    Считает количество символов, которые есть в лемме, но нет в слове.
    (Более надежно было бы считать расстояние Левенштейна, но оставим эту логику пока)
    """
    word_chars = set(word)
    return sum(1 for char in lemma if char not in word_chars)


def choose_best_lemma(word, lemmas_str):
    """
    Выбирает лучшую лемму из списка предложенных.
    Исправлено регулярное выражение для слов/лемм с не-ASCII символами.
    """
    lemmas = []
    # Используем более общее регулярное выражение для лемм:
    # ([^\s()]+) - захватывает любые символы, кроме пробелов и скобок
    # \s* - ноль или более пробелов
    # \((\d+\.?\d*)\%\) - захватывает число (целое или дробное) и знак %
    pattern = re.compile(r"([^\s()]+)\s*\((\d+\.?\d*)\%\)")
    for item in lemmas_str.split(", "):
        match = pattern.search(item.strip())
        if match:
            lemma_text = match.group(1)
            try:
                score = float(match.group(2))
                lemmas.append((lemma_text, score))
            except ValueError:
                print(f"Предупреждение: Не удалось преобразовать оценку во float для '{item}'")
                continue # Пропускаем эту лемму, если оценка не число

    if not lemmas:
        print(f"Предупреждение: Не удалось извлечь леммы из строки: '{lemmas_str}' для слова '{word}'")
        return None

    # Находим максимальный процент схожести
    max_score = 0
    # Инициализация с -1 или другим значением < 0, чтобы любая оценка была больше
    for _, score in lemmas:
        max_score = max(max_score, score)

    # Отбираем леммы с максимальным процентом
    best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]

    if len(best_lemmas) == 1:
        return best_lemmas[0]

    # Если несколько лемм с максимальным процентом - выбираем с наименьшим числом "лишних" букв в лемме
    # Эта логика может быть не идеальной, возможно, лучше выбирать самую короткую лемму или по edit distance.
    best_lemma_tuple = min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))

    return best_lemma_tuple

def process_lemmatized_file(input_file, all_file, problem_file):
    """
    Читает файл с результатами лемматизации, переносит совпадения в один файл
    и проблемы в другой.
    Исправлены регулярные выражения и логика фильтрации.
    """
    processed_count = 0
    all_count = 0
    problem_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(all_file, 'w', encoding='utf-8') as allout, \
             open(problem_file, 'w', encoding='utf-8') as probout:

            for line in infile:
                processed_count += 1
                line = line.strip() # Убираем лишние пробелы/переносы
                if not line: continue # Пропускаем пустые строки

                # 1. Случаи, когда лемма уже была найдена (Urmi или точное совпадение)
                # Используем re.match для проверки начала строки
                if re.match(r"Слово:.*,\s*Найдена лемма \(UrmiAnalyzer\):", line) or \
                   re.match(r"Слово:.*,\s*Найдена лемма \(точное совпадение\):", line):
                    allout.write(line + '\n')
                    all_count += 1
                # 2. Случаи, когда лемма уже была предложена (старый формат) - тоже в all_file
                elif "Предложенная лемма:" in line:
                     allout.write(line + '\n')
                     all_count += 1
                # 3. Случаи, когда лемм не найдено
                elif "Не найдено подходящих лемм" in line:
                    probout.write(line + '\n')
                    problem_count += 1
                # 4. Случаи, когда есть несколько предложенных лемм (по согласным)
                elif "Предложенные леммы (по согласным):" in line:
                    # Используем более общее регулярное выражение для слова
                    # Слово:\s*([^\s,]+) - Слово:, ноль или более пробелов, захват символов до пробела или запятой
                    match_word = re.search(r"Слово:\s*([^\s,]+),", line)
                    if match_word:
                        word = match_word.group(1)
                        # Извлекаем строку с леммами
                        match_lemmas = re.search(r"Предложенные леммы \(по согласным\):\s*(.+)", line)
                        if match_lemmas:
                            lemmas_str = match_lemmas.group(1)
                            best_lemma_tuple = choose_best_lemma(word, lemmas_str)

                            if best_lemma_tuple:
                               best_lemma, score = best_lemma_tuple
                               # Фильтрация: если уверенность < 63% И слово длинное (>6)
                               # Такие случаи отправляем в problem_file для ручной проверки
                               if score < 63 and len(word) > 6:
                                   probout.write(f"Слово: {word}, Низкая уверенность: {best_lemma} ({score:.1f}%)\n")
                                   problem_count += 1
                               else:
                                   # Иначе считаем лемму достаточно хорошей и пишем в all_file
                                   allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                                   all_count += 1
                            else:
                                # Если choose_best_lemma вернул None (не смог распарсить)
                                probout.write(f"Слово: {word}, Ошибка выбора лучшей леммы из: {lemmas_str}\n")
                                problem_count += 1
                        else:
                             print(f"Предупреждение: Не удалось извлечь строку лемм из: {line}")
                             probout.write(line + ' (Ошибка парсинга лемм)\n')
                             problem_count += 1
                    else:
                        print(f"Предупреждение: Не удалось извлечь слово из: {line}")
                        probout.write(line + ' (Ошибка парсинга слова)\n')
                        problem_count += 1
                # 5. Другие строки (например, ошибки из предыдущего скрипта)
                else:
                    # Решите, куда помещать неопознанные строки. Возможно, в problem_file?
                    # print(f"Неопознанный формат строки: {line}")
                    probout.write(line + ' (Неопознанный формат)\n')
                    problem_count += 1

        print(f"Обработано строк: {processed_count}")
        print(f"Записано в '{all_file}': {all_count}")
        print(f"Записано в '{problem_file}': {problem_count}")

    except FileNotFoundError:
        print(f"Ошибка: Не найден входной файл: {input_file}")
    except Exception as e:
        print(f"Критическая ошибка при обработке файла {input_file}: {e}")


if __name__ == "__main__":
    # --- Укажите правильные имена файлов ---
    input_raw_file = "prob_lemmatized_not_lemma_photo_all.txt" # Ваш файл с результатами лемматизации
    unique_file = "prob_lemmatized_not_lemma_photo_unique.txt" # Промежуточный файл без дубликатов
    all_file = "prob_lemmatized_photo_all_final.txt"        # Файл с удачно выбранными леммами
    problem_file = "prob_lemmatized_photo_problem.txt"    # Файл с проблемами/низкой уверенностью
    # -----------------------------------------

    # 1. Проверяем наличие исходного файла
    if not os.path.exists(input_raw_file):
      print(f"Ошибка: Исходный файл '{input_raw_file}' не найден.")
      exit(1) # Выходим из скрипта, если файла нет

    # 2. Удаляем дубликаты
    print(f"Удаление дубликатов из '{input_raw_file}'...")
    if remove_duplicate_lines(input_raw_file, unique_file):
        print(f"Уникальные строки сохранены в '{unique_file}'.")

        # 3. Обрабатываем файл с уникальными строками
        print(f"Обработка файла '{unique_file}'...")
        process_lemmatized_file(unique_file, all_file, problem_file)
        print(f"\nОбработка завершена.")
        print(f"Результаты с высокой уверенностью записаны в '{all_file}'.")
        print(f"Проблемные случаи или низкая уверенность записаны в '{problem_file}'.")
    else:
        print("Не удалось создать файл с уникальными строками. Обработка прервана.")



Удаление дубликатов из 'prob_lemmatized_not_lemma_photo_all.txt'...
Уникальные строки сохранены в 'prob_lemmatized_not_lemma_photo_unique.txt'.
Обработка файла 'prob_lemmatized_not_lemma_photo_unique.txt'...
Обработано строк: 13309
Записано в 'prob_lemmatized_photo_all_final.txt': 10225
Записано в 'prob_lemmatized_photo_problem.txt': 3084

Обработка завершена.
Результаты с высокой уверенностью записаны в 'prob_lemmatized_photo_all_final.txt'.
Проблемные случаи или низкая уверенность записаны в 'prob_lemmatized_photo_problem.txt'.


 по файлу с уникальными. если у слова есть вариант с совпадением согласных 80+, то проверь гласные насколько похожи. не менее 50% гласных должно быть похоже. и количество гласных не более чем на 3 может быть принижено в варианте. тогда ок, иначе в problem

 далее проверь проблемные. если они схожи между собой 70+ процентов, то выбрать наиболее частотную форму одного из них или первого по алфавиту. Так объединяться неизвестные слова. Сделать это и на этом файле и на проблемно, который на текстах.


 

ниже 2й вар программы с проверкой гласных

In [24]:
import re
import os

# === Вспомогательные функции для ГЛАСНЫХ (Добавлены) ===
# Гласные для проверки (можно расширить под ваш язык)
VOWELS = "aeiouаеёиоуыэюя"  # латиница + кириллица, добавьте свои гласные

def extract_vowels(word):
    """Возвращает строку из гласных слова в нижнем регистре."""
    return "".join([c.lower() for c in word if c.lower() in VOWELS])

def vowel_similarity(vowels1, vowels2):
    """
    Вычисляет процент совпадающих гласных между двумя строками гласных.
    Считается отношение количества общих уникальных гласных к длине меньшей строки гласных.
    """
    set1 = set(vowels1)
    set2 = set(vowels2)
    if not vowels1 or not vowels2:
        return 0.0
    common_unique_vowels = len(set1.intersection(set2))
    min_len = min(len(vowels1), len(vowels2))
    return (common_unique_vowels / min_len) * 100 if min_len > 0 else 0.0
# === Конец вспомогательных функций ===


# === Функции из вашего исходного файла paste-1.txt ===
def remove_duplicate_lines(input_file, output_file):
    """
    Читает файл, удаляет дубликаты строк и записывает результат в новый файл.
    """
    unique_lines = set()
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                unique_lines.add(line.strip()) # Удаляем лишние пробелы по краям перед добавлением
    except FileNotFoundError:
        print(f"Ошибка: Не найден файл для удаления дубликатов: {input_file}")
        return False # Сигнализируем об ошибке

    # Сортируем строки для более предсказуемого порядка (опционально)
    sorted_lines = sorted(list(unique_lines))

    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in sorted_lines:
                # Проверяем, что строка не пустая после strip()
                if line:
                    outfile.write(line + '\n') # Добавляем перенос строки обратно
        return True # Успешно
    except Exception as e:
        print(f"Ошибка при записи уникальных строк в {output_file}: {e}")
        return False

def count_mismatched_chars(word, lemma):
    """
    Считает количество символов, которые есть в лемме, но нет в слове.
    """
    word_chars = set(word)
    return sum(1 for char in lemma if char not in word_chars)


def choose_best_lemma(word, lemmas_str):
    """
    Выбирает лучшую лемму из списка предложенных.
    Исправлено регулярное выражение для слов/лемм с не-ASCII символами.
    """
    lemmas = []
    pattern = re.compile(r"([^\s()]+)\s*\((\d+\.?\d*)%\)")
    for item in lemmas_str.split(", "):
        match = pattern.search(item.strip())
        if match:
            lemma_text = match.group(1)
            try:
                score = float(match.group(2))
                lemmas.append((lemma_text, score))
            except ValueError:
                print(f"Предупреждение: Не удалось преобразовать оценку во float для '{item}'")
                continue

    if not lemmas:
        print(f"Предупреждение: Не удалось извлечь леммы из строки: '{lemmas_str}' для слова '{word}'")
        return None

    max_score = 0
    for _, score in lemmas:
        max_score = max(max_score, score)

    best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]

    if len(best_lemmas) == 1:
        return best_lemmas[0]

    best_lemma_tuple = min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))
    return best_lemma_tuple

# === Модифицированная функция process_lemmatized_file ===
def process_lemmatized_file(input_file, all_file, problem_file):
    """
    Читает файл с результатами лемматизации, применяет фильтр по гласным
    для лемм с высокой оценкой согласных, и распределяет результаты.
    """
    processed_count = 0
    all_count = 0
    problem_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(all_file, 'w', encoding='utf-8') as allout, \
             open(problem_file, 'w', encoding='utf-8') as probout:

            for line in infile:
                processed_count += 1
                line = line.strip()
                if not line: continue

                # 1. Лемма уже найдена (Urmi/точное совпадение) -> в all_file
                if re.match(r"Слово:.*,\s*Найдена лемма \(UrmiAnalyzer\):", line) or \
                   re.match(r"Слово:.*,\s*Найдена лемма \(точное совпадение\):", line):
                    allout.write(line + '\n')
                    all_count += 1
                # 2. Старый формат "Предложенная лемма:" -> в all_file
                elif "Предложенная лемма:" in line and "Предложенные леммы" not in line: # Уточнение, чтобы не пересекалось с п.4
                     allout.write(line + '\n')
                     all_count += 1
                # 3. Леммы не найдены -> в problem_file
                elif "Не найдено подходящих лемм" in line:
                    probout.write(line + '\n')
                    problem_count += 1
                # 4. Есть предложенные леммы (по согласным) -> ПРИМЕНЯЕМ ПРОВЕРКУ ГЛАСНЫХ
                elif "Предложенные леммы (по согласным):" in line:
                    match_word = re.search(r"Слово:\s*([^\s,]+),", line)
                    if match_word:
                        word = match_word.group(1)
                        match_lemmas = re.search(r"Предложенные леммы \(по согласным\):\s*(.+)", line)
                        if match_lemmas:
                            lemmas_str = match_lemmas.group(1)
                            # Выбираем сначала ЛУЧШУЮ лемму по исходным критериям
                            overall_best_lemma_tuple = choose_best_lemma(word, lemmas_str)

                            if overall_best_lemma_tuple:
                               best_lemma, score = overall_best_lemma_tuple

                               # *** НАЧАЛО ПРОВЕРКИ ГЛАСНЫХ (для score >= 80) ***
                               if score >= 80:
                                   word_vowels = extract_vowels(word)
                                   lemma_vowels = extract_vowels(best_lemma)
                                   sim = vowel_similarity(word_vowels, lemma_vowels)
                                   vowel_count_diff = len(word_vowels) - len(lemma_vowels)

                                   # Проверяем условия по гласным
                                   if sim >= 25 and vowel_count_diff <= 3:
                                       # Условия гласных выполнены -> пишем в all_file
                                       allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                                       all_count += 1
                                   else:
                                       # Условия гласных НЕ выполнены -> пишем в problem_file
                                       reason = f"совпадение гласных {sim:.1f}%"
                                       if vowel_count_diff > 3:
                                           reason += f", разница кол-ва гласных {vowel_count_diff}"
                                       probout.write(f"Слово: {word}, Лемма: {best_lemma} ({score:.1f}%), не совпало по гласным ({reason})\n")
                                       problem_count += 1
                               # *** КОНЕЦ ПРОВЕРКИ ГЛАСНЫХ ***

                               # Если score < 80, применяем старую логику фильтрации
                               elif score < 63 and len(word) > 6:
                                   # Низкая уверенность (по старой логике) -> в problem_file
                                   probout.write(f"Слово: {word}, Низкая уверенность: {best_lemma} ({score:.1f}%)\n")
                                   problem_count += 1
                               else:
                                   # Score < 80, но >= 63 ИЛИ слово короткое -> в all_file
                                   allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                                   all_count += 1
                            else:
                                # choose_best_lemma не смог выбрать -> в problem_file
                                probout.write(f"Слово: {word}, Ошибка выбора лучшей леммы из: {lemmas_str}\n")
                                problem_count += 1
                        else:
                             print(f"Предупреждение: Не удалось извлечь строку лемм из: {line}")
                             probout.write(line + ' (Ошибка парсинга лемм)\n')
                             problem_count += 1
                    else:
                        print(f"Предупреждение: Не удалось извлечь слово из: {line}")
                        probout.write(line + ' (Ошибка парсинга слова)\n')
                        problem_count += 1
                # 5. Другие/неопознанные строки -> в problem_file
                else:
                    probout.write(line + ' (Неопознанный формат)\n')
                    problem_count += 1

        print(f"Обработано строк: {processed_count}")
        print(f"Записано в '{all_file}': {all_count}")
        print(f"Записано в '{problem_file}': {problem_count}")

    except FileNotFoundError:
        print(f"Ошибка: Не найден входной файл: {input_file}")
        print("Убедитесь, что файл существует и находится в правильной папке.")
    except Exception as e:
        print(f"Критическая ошибка при обработке файла {input_file}: {e}")
        import traceback
        traceback.print_exc()


# --- Основная часть скрипта (из paste-1.txt) ---
if __name__ == "__main__":
    # --- Укажите ПРАВИЛЬНЫЕ имена файлов ---
    # Файл с результатами лемматизации (ВЫХОД предыдущего скрипта)
    input_raw_file = "prob_lemmatized_not_lemma_photo_all.txt"
    # Промежуточный файл БЕЗ ДУБЛИКАТОВ (ВХОД для process_lemmatized_file)
    unique_file = "prob_lemmatized_not_lemma_photo_unique.txt"
    # ВЫХОД: Файл с удачно выбранными леммами (после всех проверок)
    all_file = "prob_lemmatized_photo_all_final.txt"
    # ВЫХОД: Файл с проблемами/низкой уверенностью/несовпадением гласных
    problem_file = "prob_lemmatized_photo_problem.txt"
    # -----------------------------------------

    # 1. Проверяем наличие исходного файла с результатами лемматизации
    if not os.path.exists(input_raw_file):
      print(f"Ошибка: Исходный файл '{input_raw_file}' не найден.")
      exit(1)

    # 2. Удаляем дубликаты из исходного файла
    print(f"Удаление дубликатов из '{input_raw_file}'...")
    if remove_duplicate_lines(input_raw_file, unique_file):
        print(f"Уникальные строки сохранены в '{unique_file}'.")

        # 3. Обрабатываем файл с уникальными строками (теперь с проверкой гласных)
        print(f"Обработка файла '{unique_file}' (с проверкой гласных)...")
        # Убедимся, что process_lemmatized_file использует unique_file как входной
        process_lemmatized_file(unique_file, all_file, problem_file)
        print(f"\nОбработка завершена.")
        print(f"Результаты с высокой уверенностью (или прошедшие проверку гласных) записаны в '{all_file}'.")
        print(f"Проблемные случаи / несовпадение гласных / низкая уверенность записаны в '{problem_file}'.")
    else:
        print("Не удалось создать файл с уникальными строками. Обработка прервана.")



Удаление дубликатов из 'prob_lemmatized_not_lemma_photo_all.txt'...
Уникальные строки сохранены в 'prob_lemmatized_not_lemma_photo_unique.txt'.
Обработка файла 'prob_lemmatized_not_lemma_photo_unique.txt' (с проверкой гласных)...
Обработано строк: 13309
Записано в 'prob_lemmatized_photo_all_final.txt': 7265
Записано в 'prob_lemmatized_photo_problem.txt': 6044

Обработка завершена.
Результаты с высокой уверенностью (или прошедшие проверку гласных) записаны в 'prob_lemmatized_photo_all_final.txt'.
Проблемные случаи / несовпадение гласных / низкая уверенность записаны в 'prob_lemmatized_photo_problem.txt'.


Конечный код (3й вариант. С доп проверкой гласных + префиксов)

In [5]:
import re
import os
import traceback # For detailed error printing

# === Вспомогательные функции для ГЛАСНЫХ ===
VOWELS = "aeiouаеёиоуыэюя"

# === НОВОЕ: Множество согласных (расширьте при необходимости) ===
# Включаем латиницу и основные кириллические согласные
CONSONANTS = set("bcdfghjklmnpqrstvwxyzбвгджзйклмнпрстфхцчшщ")

def extract_vowels(word):
    """Возвращает строку из гласных слова в нижнем регистре."""
    return "".join([c.lower() for c in word if c.lower() in VOWELS])

def vowel_similarity(vowels1, vowels2):
    """Вычисляет процент общих уникальных гласных к min длине."""
    set1 = set(vowels1)
    set2 = set(vowels2)
    if not vowels1 or not vowels2: return 0.0
    common_unique_vowels = len(set1.intersection(set2))
    min_len = min(len(vowels1), len(vowels2))
    return (common_unique_vowels / min_len) * 100 if min_len > 0 else 0.0

# === Функции из вашего исходного скрипта ===
def remove_duplicate_lines(input_file, output_file):
    """Читает файл, удаляет дубликаты строк и записывает результат."""
    unique_lines = set()
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile: unique_lines.add(line.strip())
    except FileNotFoundError:
        print(f"Ошибка: Не найден файл для удаления дубликатов: {input_file}")
        return False
    sorted_lines = sorted(list(unique_lines))
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in sorted_lines:
                if line: outfile.write(line + '\n')
        return True
    except Exception as e:
        print(f"Ошибка при записи уникальных строк в {output_file}: {e}")
        return False

def count_mismatched_chars(word, lemma):
    """Считает символы леммы, отсутствующие в слове."""
    word_chars = set(word)
    return sum(1 for char in lemma if char not in word_chars)

def choose_best_lemma(word, lemmas_str):
    """Выбирает лучшую лемму из строки с предложениями."""
    lemmas = []
    pattern = re.compile(r"([^\s()]+)\s*\((\d+\.?\d*)%\)")
    # Добавим проверку, что lemmas_str не пустой
    if not lemmas_str:
        print(f"Предупреждение: Пустая строка лемм для слова '{word}'")
        return None
    for item in lemmas_str.split(", "):
        match = pattern.search(item.strip())
        if match:
            lemma_text = match.group(1)
            try: score = float(match.group(2))
            except ValueError:
                print(f"Предупреждение: Не удалось преобразовать оценку во float для '{item}' в строке '{lemmas_str}'")
                continue
            lemmas.append((lemma_text, score))

    if not lemmas:
        # Убрано предупреждение отсюда, т.к. может быть вызвано пустым списком после парсинга
        return None

    try:
        max_score = max(score for _, score in lemmas)
    except ValueError: # Если список lemmas пуст
         print(f"Предупреждение: Не найдено валидных лемм в строке '{lemmas_str}' для слова '{word}'")
         return None

    best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]

    if not best_lemmas: # На всякий случай
         return None
    if len(best_lemmas) == 1:
        return best_lemmas[0]

    # Если несколько лемм с макс. оценкой - выбираем по мин. числу "лишних" букв
    try:
        best_lemma_tuple = min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))
        return best_lemma_tuple
    except Exception as e: # Ловим возможные ошибки при вычислении ключа
        print(f"Ошибка при выборе лучшей леммы (min) для слова '{word}' из {best_lemmas}: {e}")
        return best_lemmas[0] # Возвращаем первую из лучших как запасной вариант

# === Модифицированная функция process_lemmatized_file ===
def process_lemmatized_file(input_file, all_file, problem_file):
    """
    Читает файл с леммами, применяет фильтр по гласным и префиксам,
    распределяет результаты.
    """
    processed_count = 0
    all_count = 0
    problem_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(all_file, 'w', encoding='utf-8') as allout, \
             open(problem_file, 'w', encoding='utf-8') as probout:

            for line_num, line in enumerate(infile, 1): # Добавим номер строки
                processed_count += 1
                line = line.strip()
                if not line: continue

                # Обработка строк с уже найденными леммами или без лемм
                if re.match(r"Слово:.*,\s*Найдена лемма \(UrmiAnalyzer\):", line) or \
                   re.match(r"Слово:.*,\s*Найдена лемма \(точное совпадение\):", line) or \
                   ("Предложенная лемма:" in line and "Предложенные леммы" not in line):
                    allout.write(line + '\n')
                    all_count += 1
                    continue
                elif "Не найдено подходящих лемм" in line:
                    probout.write(line + '\n')
                    problem_count += 1
                    continue

                # --- Обработка строк с "Предложенные леммы (по согласным):" ---
                elif "Предложенные леммы (по согласным):" in line:
                    match_word = re.search(r"Слово:\s*([^\s,]+),", line)
                    if not match_word:
                        print(f"Предупреждение (строка {line_num}): Не удалось извлечь слово из: {line}")
                        probout.write(line + ' (Ошибка парсинга слова)\n')
                        problem_count += 1
                        continue
                    word = match_word.group(1)

                    match_lemmas = re.search(r"Предложенные леммы \(по согласным\):\s*(.+)", line)
                    if not match_lemmas:
                         print(f"Предупреждение (строка {line_num}): Не удалось извлечь строку лемм из: {line}")
                         probout.write(line + ' (Ошибка парсинга лемм)\n')
                         problem_count += 1
                         continue
                    lemmas_str = match_lemmas.group(1)

                    # --- НОВОЕ: Проверка префикса и альтернативный поиск ---
                    final_lemma_tuple = None
                    prefix_check_applied = False

                    if word and len(word) > 1 and word[0].lower() in 'dlbв' and word[1].lower() in CONSONANTS:
                        prefix_check_applied = True
                        modified_word = word[1:]
                        # Ищем лучшую лемму для слова БЕЗ первой буквы
                        alternative_lemma_tuple = choose_best_lemma(modified_word, lemmas_str)
                        if alternative_lemma_tuple:
                            final_lemma_tuple = alternative_lemma_tuple
                            # print(f"  [DEBUG] Префикс! Для '{word}' выбран альт. вариант по '{modified_word}': {final_lemma_tuple}") # Отладка
                        else:
                            # Если альтернативный поиск не дал результата, возвращаемся к обычному
                            final_lemma_tuple = choose_best_lemma(word, lemmas_str)
                            # print(f"  [DEBUG] Префикс! Альт. поиск для '{modified_word}' не дал результата, выбран обычный: {final_lemma_tuple}") # Отладка
                    else:
                        # Обычный выбор лучшей леммы, если префикс не подходит
                        final_lemma_tuple = choose_best_lemma(word, lemmas_str)

                    # --- Конец проверки префикса ---

                    # --- Дальнейшая обработка с использованием final_lemma_tuple ---
                    if final_lemma_tuple:
                       best_lemma, score = final_lemma_tuple

                       # *** ПРОВЕРКА ГЛАСНЫХ (для score >= 80) ***
                       if score >= 80:
                           # Используем ОРИГИНАЛЬНОЕ слово для сравнения гласных
                           word_vowels = extract_vowels(word)
                           lemma_vowels = extract_vowels(best_lemma)
                           sim = vowel_similarity(word_vowels, lemma_vowels)
                           vowel_count_diff = len(word_vowels) - len(lemma_vowels)

                           # Условия гласных: схожесть >= 25% И разница <= 3
                           if sim >= 25 and vowel_count_diff <= 3:
                               allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                               all_count += 1
                           else:
                               reason = f"совпадение гласных {sim:.1f}%"
                               if vowel_count_diff > 3: reason += f", разница кол-ва гласных {vowel_count_diff}"
                               probout.write(f"Слово: {word}, Лемма: {best_lemma} ({score:.1f}%), не совпало по гласным ({reason})\n")
                               problem_count += 1
                       # *** КОНЕЦ ПРОВЕРКИ ГЛАСНЫХ ***

                       # --- Обработка для score < 80 (старая логика) ---
                       elif score < 63 and len(word) > 6:
                           probout.write(f"Слово: {word}, Низкая уверенность: {best_lemma} ({score:.1f}%)\n")
                           problem_count += 1
                       else:
                           # Score < 80, но >= 63 ИЛИ слово короткое <= 6
                           allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                           all_count += 1
                    else:
                        # Если choose_best_lemma (основной или альтернативный) вернул None
                        probout.write(f"Слово: {word}, Ошибка выбора лучшей леммы из: {lemmas_str}\n")
                        problem_count += 1

                # --- Обработка других/неопознанных строк ---
                else:
                    # Записываем неопознанные строки в файл проблем
                    probout.write(line + ' (Неопознанный формат)\n')
                    problem_count += 1

        print(f"\nОбработано строк: {processed_count}")
        print(f"Записано в '{all_file}': {all_count}")
        print(f"Записано в '{problem_file}': {problem_count}")

    except FileNotFoundError:
        print(f"Критическая ошибка: Не найден входной файл: {input_file}")
    except Exception as e:
        print(f"Критическая ошибка при обработке файла {input_file} (около строки {line_num}): {e}")
        traceback.print_exc()


# --- Основная часть скрипта ---
if __name__ == "__main__":
    # --- Имена файлов ---
    input_raw_file = "prob_lemmatized_not_lemma_photo_all.txt"
    unique_file = "prob_lemmatized_not_lemma_photo_unique.txt"
    all_file = "prob_lemmatized_photo_all_final.txt"
    problem_file = "prob_lemmatized_photo_problem.txt"
    # -----------------------------------------

    if not os.path.exists(input_raw_file):
      print(f"Ошибка: Исходный файл '{input_raw_file}' не найден.")
      exit(1)

    print(f"Удаление дубликатов из '{input_raw_file}'...")
    if remove_duplicate_lines(input_raw_file, unique_file):
        print(f"Уникальные строки сохранены в '{unique_file}'.")
        print(f"Обработка файла '{unique_file}' (с проверкой гласных и префиксов)...")
        process_lemmatized_file(unique_file, all_file, problem_file)
        print(f"\nОбработка завершена.")
        print(f"Результаты с высокой уверенностью / прошедшие проверки записаны в '{all_file}'.")
        print(f"Проблемные случаи / низкая уверенность / несовпадение гласных записаны в '{problem_file}'.")
    else:
        print("Не удалось создать файл с уникальными строками. Обработка прервана.")



Удаление дубликатов из 'prob_lemmatized_not_lemma_photo_all.txt'...
Уникальные строки сохранены в 'prob_lemmatized_not_lemma_photo_unique.txt'.
Обработка файла 'prob_lemmatized_not_lemma_photo_unique.txt' (с проверкой гласных и префиксов)...

Обработано строк: 13309
Записано в 'prob_lemmatized_photo_all_final.txt': 7265
Записано в 'prob_lemmatized_photo_problem.txt': 6044

Обработка завершена.
Результаты с высокой уверенностью / прошедшие проверки записаны в 'prob_lemmatized_photo_all_final.txt'.
Проблемные случаи / низкая уверенность / несовпадение гласных записаны в 'prob_lemmatized_photo_problem.txt'.


In [ ]:
# подчищенный

import re
import os
import traceback

# глобальные строчные константы
VOWELS = "aeiouаеёиоуыэюя"
CONSONANTS = set("bcdfghjklmnpqrstvwxyzбвгджзйклмнпрстфхцчшщ")

# ФУНКЦИИ

# для извлечения гласных из слова
def extract_vowels(word):
    return "".join([c.lower() for c in word if c.lower() in VOWELS])

# для расчета схожести гласных
def vowel_similarity(vowels1, vowels2):
    set1 = set(vowels1)
    set2 = set(vowels2)
    if not vowels1 or not vowels2: return 0.0
    common_unique_vowels = len(set1.intersection(set2))
    min_len = min(len(vowels1), len(vowels2))
    return (common_unique_vowels / min_len) * 100 if min_len > 0 else 0.0

# для удаления дубликатов строк
def remove_duplicate_lines(input_file, output_file):
    unique_lines = set()
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile: unique_lines.add(line.strip())
    except FileNotFoundError:
        print(f"Ошибка: Не найден файл: {input_file}")
        return False
    
    sorted_lines = sorted(list(unique_lines))
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in sorted_lines:
                if line: outfile.write(line + '\n')
        return True
    except Exception as e:
        print(f"Ошибка записи: {output_file}: {e}")
        return False

# ОСНОВНОЙ
# для подсчета несовпадающих символов
def count_mismatched_chars(word, lemma):
    word_chars = set(word)
    return sum(1 for char in lemma if char not in word_chars)

# для выбора оптимальной леммы
def choose_best_lemma(word, lemmas_str):
    lemmas = []
    pattern = re.compile(r"([^\s()]+)\s*\((\d+\.?\d*)%\)")
    
    if not lemmas_str:
        return None

    #парсинг строки с вариантами лемм
    for item in lemmas_str.split(", "):
        match = pattern.search(item.strip())
        if match:
            lemma_text = match.group(1)
            try: score = float(match.group(2))
            except ValueError: continue
            lemmas.append((lemma_text, score))

    if not lemmas: return None

    #Выбор леммы с макс %
    try:
        max_score = max(score for _, score in lemmas)
        best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]
        
        if len(best_lemmas) == 1:
            return best_lemmas[0]
            
        # проверка если с одинаковым %
        return min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))
    except:
        return None

# ПРОЦЕСС

#основная функция обработки файла с леммами
def process_lemmatized_file(input_file, all_file, problem_file):
    processed_count = 0
    all_count = 0
    problem_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(all_file, 'w', encoding='utf-8') as allout, \
             open(problem_file, 'w', encoding='utf-8') as probout:

            # каждая строка
            for line_num, line in enumerate(infile, 1):
                processed_count += 1
                line = line.strip()
                if not line: continue

                # разные типы строк
                if re.match(r"Слово:.*,\s*Найдена лемма \(UrmiAnalyzer\):", line) or \
                   re.match(r"Слово:.*,\s*Найдена лемма \(точное совпадение\):", line) or \
                   ("Предложенная лемма:" in line and "Предложенные леммы" not in line):
                    allout.write(line + '\n')
                    all_count += 1
                    continue
                elif "Не найдено подходящих лемм" in line:
                    probout.write(line + '\n')
                    problem_count += 1
                    continue

                #Обработка предложенных лемм
                elif "Предложенные леммы (по согласным):" in line:
                    # Парсинг слова и вариантов лемм
                    match_word = re.search(r"Слово:\s*([^\s,]+),", line)
                    if not match_word:
                        probout.write(line + ' (Ошибка парсинга слова)\n')
                        problem_count += 1
                        continue
                    word = match_word.group(1)

                    # Доп обработка
                    match_lemmas = re.search(r"Предложенные леммы \(по согласным\):\s*(.+)", line)
                    if not match_lemmas:
                         probout.write(line + ' (Ошибка парсинга лемм)\n')
                         problem_count += 1
                         continue

                    # выбор финальной леммы
                    lemmas_str = match_lemmas.group(1)
                    final_lemma_tuple = None

                    # Спец обработка для слов с определенными ПРЕФИКСАМИ
                    if word and len(word) > 1 and word[0].lower() in 'dlbв' and word[1].lower() in CONSONANTS:
                        modified_word = word[1:]
                        alternative_lemma_tuple = choose_best_lemma(modified_word, lemmas_str)
                        final_lemma_tuple = alternative_lemma_tuple if alternative_lemma_tuple else choose_best_lemma(word, lemmas_str)
                    else:
                        final_lemma_tuple = choose_best_lemma(word, lemmas_str)

                    #Проверка выбранной леммы  
                    if final_lemma_tuple:
                       best_lemma, score = final_lemma_tuple

                       #Проверка гласных для больших %
                       if score >= 80:
                           word_vowels = extract_vowels(word)
                           lemma_vowels = extract_vowels(best_lemma)
                           sim = vowel_similarity(word_vowels, lemma_vowels)
                           vowel_count_diff = len(word_vowels) - len(lemma_vowels)

                           # фильтрует по гласным
                           if sim >= 25 and vowel_count_diff <= 3:
                               allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                               all_count += 1
                           else:
                               reason = f"совпадение гласных {sim:.1f}%"
                               if vowel_count_diff > 3: reason += f", разница гласных {vowel_count_diff}"
                               probout.write(f"Слово: {word}, Лемма: {best_lemma} ({score:.1f}%), не совпало по гласным ({reason})\n")
                               problem_count += 1

                       # Обработывает низкокачественные леммы
                       elif score < 63 and len(word) > 6:
                           probout.write(f"Слово: {word}, Низкая уверенность: {best_lemma} ({score:.1f}%)\n")
                           problem_count += 1
                       else:
                           allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                           all_count += 1
                    else:
                        probout.write(f"Слово: {word}, Ошибка выбора леммы из: {lemmas_str}\n")
                        problem_count += 1

                else:
                    probout.write(line + ' (Неопознанный формат)\n')
                    problem_count += 1

        print(f"\nОбработано строк: {processed_count}")
        print(f"Успешные леммы: {all_count}")
        print(f"Проблемные случаи: {problem_count}")

    except FileNotFoundError:
        print(f"Ошибка: Файл не найден: {input_file}")
    except Exception as e:
        print(f"Критическая ошибка: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    input_raw_file = "prob_lemmatized_not_lemma_photo_all.txt"
    unique_file = "prob_lemmatized_not_lemma_photo_unique.txt"
    all_file = "prob_lemmatized_photo_all_final.txt"
    problem_file = "prob_lemmatized_photo_problem.txt"

    if not os.path.exists(input_raw_file):
      print(f"Ошибка: Файл не найден: {input_raw_file}")
      exit(1)

    print(f"Удаление дубликатов...")
    if remove_duplicate_lines(input_raw_file, unique_file):
        print(f"Обработка файла...")
        process_lemmatized_file(unique_file, all_file, problem_file)
        print(f"\nРезультаты:")
        print(f"Успешные леммы: {all_file}")
        print(f"Проблемные случаи: {problem_file}")
    else:
        print("Ошибка обработки")


#### **6. Объединение проблемных слов**

In [6]:
import re
from collections import defaultdict, Counter
from difflib import SequenceMatcher
import os 

input_problem_file = 'prob_lemmatized_photo_problem.txt'
output_file = 'photo_problems.txt' 

# для вычисления похожести строк
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio() * 100

# Читка проблем и их лемм из файла
def read_problem_words(input_file):
    problem_words = []
    pattern_word = re.compile(r"Слово: ([^,]+),")
    pattern_lemma = re.compile(r"Лемма: ([^\(]+) \((\d+\.?\d*)%\)")
    pattern_line = re.compile(r"Слово: ([^,]+), Лемма: ([^\(]+) \((\d+\.?\d*)%\),?")

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                #поиск слова и леммы
                m_word = pattern_word.search(line)
                if m_word:
                    word = m_word.group(1)
                    lemmas = []
                    m_lemma = pattern_lemma.findall(line)
                    if m_lemma:
                        for lm in m_lemma:
                            lemmas.append((lm[0].strip(), float(lm[1])))
                    else:
                        # Если леммы нет, поискать в след строке
                        m_line = pattern_line.search(line)
                        if m_line:
                            word = m_line.group(1)
                            lemmas = [(m_line.group(2).strip(), float(m_line.group(3)))]
                        else:
                            lemmas = []
                    problem_words.append({"word": word, "lemmas": lemmas, "line": line})
    except FileNotFoundError:
        print(f"Ошибка: Файл '{input_file}' не найден.")
        return None # если ошибка
    except Exception as e:
        print(f"Ошибка при чтении файла '{input_file}': {e}")
        return None

    return problem_words


#Объединение схожих слов!
def cluster_similar_words(problem_words, similarity_threshold=70):
    clusters = []
    used_indices = set()

    for i, pw1 in enumerate(problem_words):
        if i in used_indices:
            continue
        cluster = [pw1]
        used_indices.add(i)
        for j, pw2 in enumerate(problem_words[i+1:], start=i+1):
            if j in used_indices:
                continue
            sim = similarity(pw1['word'], pw2['word'])
            if sim >= similarity_threshold:
                cluster.append(pw2)
                used_indices.add(j)
        clusters.append(cluster)
    return clusters

# f чтобы выбрать наиболее частотную или по первой по алфавиту форму
def choose_representative(cluster):
    words = [item['word'] for item in cluster]
    freq = Counter(words)
    max_freq = max(freq.values())
    candidates = [w for w, c in freq.items() if c == max_freq]
    return candidates[0]

# Объединение лемм из кластера
def merge_lemmas(cluster):
    lemma_scores = defaultdict(float)
    for item in cluster:
        for lemma, score in item['lemmas']:
            # сохранять макс оценку для леммы
            lemma_scores[lemma] = max(lemma_scores[lemma], score)
    # Сортирует леммы по УБЫВАНИЮ оценки (т.е сначала лучшие)
    sorted_lemmas = sorted(lemma_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_lemmas

# ДЛЯ ИТОГ записи
def format_results(clusters):
    merged_results = []
    for cluster in clusters:
        rep_word = choose_representative(cluster)
        merged_lemmas = merge_lemmas(cluster)
        lemmas_str = ", ".join([f"{lemma} ({score:.1f}%)" for lemma, score in merged_lemmas])
        merged_results.append(f"Слово: {rep_word}, Предложенные леммы: {lemmas_str}")
    return merged_results


# ОСНОВ ЧАСТЬ
if __name__ == "__main__":
    if not os.path.exists(input_problem_file):
        print(f"Ошибка: Файл '{input_problem_file}' не найден.")
        exit(1)

    print(f"Чтение данных из '{input_problem_file}'...")
    problem_words = read_problem_words(input_problem_file)

    if problem_words is None: # на ошибку
        print("Чтение файла прервано из-за ошибки.")
        exit(1)

    # кластеризуем схожие слова
    print("Кластеризация схожих слов...")
    clusters = cluster_similar_words(problem_words)

    # формируем резы
    print("Формирование результатов...")
    formatted_results = format_results(clusters)

    # запись в файл
    print(f"Запись результатов в '{output_file}'...")
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in formatted_results:
                outfile.write(line + '\n')

        print(f"Объединенные результаты сохранены в файл '{output_file}'.")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_file}': {e}")


Чтение данных из 'prob_lemmatized_photo_problem.txt'...
Кластеризация схожих слов...
Формирование результатов...
Запись результатов в 'photo_problems.txt'...
Объединенные результаты сохранены в файл 'photo_problems.txt'.


#### **7. Восстановление предложений**

In [10]:
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# ИМЕННЫХ СУЩНОСТИ загрузка
NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()
try:
    with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            entity = line.strip()
            if entity:
                named_entities.add(entity.lower())  # сохраняем в нижнем регистре для сравнения
    print(f"Загружено {len(named_entities)} уникальных именованных сущностей из '{NAMED_ENTITIES_FILE}'.")
except FileNotFoundError:
    print(f"Предупреждение: Файл '{NAMED_ENTITIES_FILE}' не найден. Проверка на именованные сущности будет отключена.")
except Exception as e:
    print(f"Ошибка при чтении файла '{NAMED_ENTITIES_FILE}': {e}")



input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt'
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final.txt'




def similarity(a, b):
    """Схожесть строк в % (без учета регистра)"""
    if not a or not b: return 0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100

def load_prob_final_lemmas(filename):
    """Загружает леммы из prob_final_file в словарь {слово_lower: лемма}."""
    start_time = time.time()
    print(f"Загрузка лемм из '{filename}'...")
    lemma_map = {}
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    lemma = match.group(2).strip()
                    if word and lemma:
                        lemma_map[word.lower()] = lemma 
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} лемм из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск в нем будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """Загружает 'Слово:' из photo_problems.txt в список."""
    start_time = time.time()
    print(f"Загрузка проблемных слов из '{filename}'...")
    word_list = []
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    if word:
                        word_list.append(word)
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} слов из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск схожих слов будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """
    Ищет первое слово в word_list (дл > 3, схожесть >= threshold). приоритет словам со строчной буквы
    """
    if not target_word: return None
    best_match_lower = None
    best_match_upper = None
    for word in word_list:
        if len(word) > 3:
            sim_score = similarity(target_word, word) # сравнение уже без регистра
            if sim_score >= threshold:
                # Проверяем регистр кандидата 
                if not word or not word[0].isalpha() or word[0].islower():
                    if best_match_lower is None:
                        best_match_lower = word
                        return best_match_lower # Нашли лучший - выходим
                elif best_match_upper is None:
                     best_match_upper = word # запоминаем первый прописной
    return best_match_upper # возвращаеМ прописной, если строчный не найден


def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
     """
     если лемма из основного файла не подходит:
     Иерархию: именованная сущность -> prob_final -> problems -> token -> word.
     """
     if not word:
         return token or "" # возвращаем токен или пустую строку, если слова нет

     # 1/ Проверяем на именован сущ
     if word.lower() in named_entities:
         # Если это именованная сущность, возвращаем ОРИГИНАЛЬНОЕ слово 
         return word

     lemma_to_add = None #для дальнейшего

     # 2\ Поиск в prob_final_lemmas
     search_key_prob = word.lower()
     found_lemma_in_prob = prob_lemmas.get(search_key_prob)

     if found_lemma_in_prob:
         # Если исходное Слово: было с большой буквы, используем его
         if word and word[0].isupper():
             lemma_to_add = word
         else:
             lemma_to_add = found_lemma_in_prob
         return lemma_to_add

     #  3\ Поиск схожего в clustered_problem_words
     search_target_problems = word.lower() if is_first_word else word
     found_similar = find_similar_word(search_target_problems, problem_words, 70)

     if found_similar:
         lemma_to_add = found_similar
         return lemma_to_add

     # 4\ Обратно в Токен
     if token:
         lemma_to_add = token
         return lemma_to_add

     # 5\ Обратно в Слово
     lemma_to_add = word
     return lemma_to_add


# ОСНОВА

# Проверка файлов и выгрузка
if not os.path.exists(input_main_file):
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)

print("--- Загрузка вспомогательных данных ---")
overall_start_time = time.time()
print("--- Загрузка завершена ---")

# процесс
results = []
current_sentence_text = None
current_lemmas = []
current_word = None
current_token = None
is_first_word_in_sentence = False # Флаг 1го слова
processed_lines_count = 0
processed_sentences_count = 0
last_print_time = time.time()
line_num = 0

print(f"--- Обработка основного файла '{input_main_file}' ---")
try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line_num, line in enumerate(infile, 1):
            processed_lines_count += 1
            line = line.strip()

            current_time = time.time()
            if current_time - last_print_time > 5: # индикатор прогресса
                print(f"Обработано строк: {processed_lines_count}, найдено предложений: {processed_sentences_count}...")
                last_print_time = current_time

            if not line: continue #Пропускаем пустые строки

            # Новое предложение
            if line.startswith("Предложение:"):
                if current_sentence_text is not None:
                    #завершаем последнее слово пред. предложения
                    if current_word:
                        # флаг is_first_word_in_sentence
                        final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                        if final_lemma: current_lemmas.append(final_lemma)
                    # сохраняем результат, если есть леммы
                    if current_lemmas:
                        results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
                        processed_sentences_count += 1

                # новое предложение
                try:
                    current_sentence_text = line.split(": ", 1)[1]
                except IndexError:
                    print(f"Предупреждение (строка {line_num}): Не удалось извлечь текст предложения: {line}")
                    current_sentence_text = ""
                current_lemmas = []
                current_word = None
                current_token = None
                is_first_word_in_sentence = True # Устанавливаем флаг для нового предложения

            # Новое слово
            elif line.startswith("Слово:"):
                 #завершаем предыдущее слово, если оно не было обработано строкой "Лемма:"
                 if current_word:
                     # флаг is_first_word_in_sentence
                     final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                     if final_lemma: current_lemmas.append(final_lemma)
                     # сбрасываем флаг после обработки, если это было первое слово
                     if is_first_word_in_sentence:
                         is_first_word_in_sentence = False

                 #Запоминаем новое слово
                 try:
                     current_word = line.split(": ", 1)[1]
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь слово из: {line}")
                     current_word = None
                 current_token = None # сбрасываем токен для нового слова

            # ТОКЕН
            elif line.startswith("Токен:") and current_word:
                 try:
                     current_token = line.split(": ", 1)[1]
                     if not current_token: current_token = current_word # Если токен пуст
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь токен из: {line}")
                     current_token = current_word

            # ЛЕММА
            elif line.startswith("Лемма:") and current_word:
                lemma_from_file = ""
                try:
                    if ": " in line:
                        lemma_part = line.split(": ", 1)[1]
                        lemma_from_file = lemma_part.split('/', 1)[0].strip()
                    else: lemma_from_file = ""
                except IndexError: lemma_from_file = ""

                # ПО ИЕРАРХИИ
                lemma_to_add = None
                # 1: Лемма из файла (если >= 4 символов)
                if lemma_from_file and len(lemma_from_file) >= 4:
                    lemma_to_add = lemma_from_file
                else:
                    #  0 и Шаги 2-5 (с учетом флага is_first_word)
                    # Проверка на named_entity
                    lemma_to_add = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)

                if lemma_to_add: current_lemmas.append(lemma_to_add)

                # сбрасываем флаг первого слова, если он еще стоял (даже если сработал шаг 1, слово уже не первое)
                if is_first_word_in_sentence:
                    is_first_word_in_sentence = False

                # Сбрасываем состояние слова, тк оно обработано
                current_word = None
                current_token = None

    # Обработка последнего предложения
    if current_sentence_text is not None:
        if current_word: # самое последнее слово
            # флаг is_first_word_in_sentence
            final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
            if final_lemma: current_lemmas.append(final_lemma)
        # если есть лемма, сохраняем
        if current_lemmas:
             results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
             processed_sentences_count += 1

except FileNotFoundError:
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)
except Exception as e:
    print(f"Критическая ошибка при обработке основного файла (около строки {line_num}): {e}")
    traceback.print_exc()
    exit(1)

processing_time = time.time() - overall_start_time
print(f"--- Обработка основного файла завершена за {processing_time:.2f} сек. Обработано строк: {processed_lines_count} ---")

# 4.Запись реза
print(f"--- Запись результатов в '{output_final_sentences_file}' ---")
write_start_time = time.time()
if results:
    try:
        with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
            for item in results:
                lemma_string = ' '.join(map(str, item['lemmas']))
                outfile.write(lemma_string.strip() + '\n')
        write_time = time.time() - write_start_time
        print(f"Запись завершена за {write_time:.2f} сек. Сохранено предложений: {len(results)}")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_final_sentences_file}': {e}")
else:
    print("Нет данных для записи в выходной файл.")

total_time = time.time() - overall_start_time
print(f"--- Скрипт завершил работу за {total_time:.2f} сек. ---")


Загружено 861 уникальных именованных сущностей из 'named_entities.txt'.
--- Загрузка вспомогательных данных ---
--- Загрузка завершена ---
--- Обработка основного файла 'lemmatized_not_punct_all.txt' ---
Обработано строк: 1691, найдено предложений: 13...
Обработано строк: 3715, найдено предложений: 41...
Обработано строк: 5723, найдено предложений: 63...
Обработано строк: 7517, найдено предложений: 86...
Обработано строк: 9601, найдено предложений: 112...
Обработано строк: 11721, найдено предложений: 134...
Обработано строк: 13885, найдено предложений: 164...
Обработано строк: 17921, найдено предложений: 212...
Обработано строк: 22973, найдено предложений: 276...
Обработано строк: 27331, найдено предложений: 327...
Обработано строк: 31689, найдено предложений: 378...
Обработано строк: 35385, найдено предложений: 424...
Обработано строк: 39677, найдено предложений: 470...
Обработано строк: 44391, найдено предложений: 539...
Обработано строк: 49071, найдено предложений: 601...
Обработано

то, что ниже - лучшее для составления предложений без удаления именов сущностей

In [ ]:
# почищенное от комментов

import re
import os
from difflib import SequenceMatcher
import traceback
import time

NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()

# ЗАГРУЗКА ИМЕННЫХ СУЩНОСТЕЙ 
try:
    with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            entity = line.strip()
            if entity:
                named_entities.add(entity.lower())
    print(f"Загружено {len(named_entities)} уникальных именованных сущностей.")
except FileNotFoundError:
    print(f"Файл '{NAMED_ENTITIES_FILE}' не найден.")
except Exception as e:
    print(f"Ошибка: {e}")

input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt'
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final.txt'

# F
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100 if a and b else 0

def load_prob_final_lemmas(filename):
    lemma_map = {}
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)", line)
                if match:
                    lemma_map[match.group(1).lower()] = match.group(2).strip()
    except Exception as e:
        print(f"Ошибка загрузки лемм: {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    word_list = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,", line)
                if match:
                    word_list.append(match.group(1))
    except Exception as e:
        print(f"Ошибка загрузки проблемных слов: {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    best_match = None
    for word in word_list:
        if len(word) > 3 and similarity(target_word, word) >= threshold:
            if not best_match or word[0].islower():
                best_match = word
                if word[0].islower():
                    break
    return best_match

# ОСНОВНОЕ
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
    if not word:
        return token or ""

    if word.lower() in named_entities:
        return word

    lemma = prob_lemmas.get(word.lower())
    if lemma:
        return word if word[0].isupper() else lemma

    similar = find_similar_word(word.lower() if is_first_word else word, problem_words)
    if similar:
        return similar

    return token or word


results = []
current_sentence = []
current_word = None
current_token = None
is_first_word = False

try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            if line.startswith("Предложение:"):
                if current_sentence:
                    results.append({"sentence": current_sentence_text, "lemmas": current_sentence})
                current_sentence_text = line.split(": ", 1)[1]
                current_sentence = []
                is_first_word = True

            elif line.startswith("Слово:"):
                if current_word:
                    lemma = apply_fallback_logic(current_word, current_token, 
                                               prob_final_lemmas, clustered_problem_words, 
                                               is_first_word)
                    current_sentence.append(lemma)
                current_word = line.split(": ", 1)[1]
                current_token = None
                is_first_word = False

            elif line.startswith("Токен:") and current_word:
                current_token = line.split(": ", 1)[1] or current_word

            elif line.startswith("Лемма:") and current_word:
                lemma = line.split(": ", 1)[1].split('/', 1)[0].strip()
                current_sentence.append(lemma if len(lemma) >= 4 else 
                                      apply_fallback_logic(current_word, current_token,
                                                         prob_final_lemmas, clustered_problem_words,
                                                         is_first_word))
                current_word = None

    if current_sentence:
        results.append({"sentence": current_sentence_text, "lemmas": current_sentence})

except Exception as e:
    print(f"Критическая ошибка: {e}")
    exit(1)

# сохранение резов
try:
    with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
        for item in results:
            outfile.write(' '.join(item['lemmas']) + '\n')
except Exception as e:
    print(f"Ошибка записи: {e}")

print("Обработка завершена.")

In [17]:
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# ========== ЗАГРУЗКА ИМЕННЫХ СУЩНОСТЕЙ ==========
# Имя файла с именованными сущностями (одно слово/сущность на строку)
NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()
named_entities_original_case = {} # Словарь для хранения оригинального регистра
try:
    with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            entity = line.strip()
            if entity:
                entity_lower = entity.lower()
                named_entities.add(entity_lower)
                # Сохраняем оригинальный регистр (первый встреченный вариант)
                if entity_lower not in named_entities_original_case:
                    named_entities_original_case[entity_lower] = entity
    print(f"Загружено {len(named_entities)} уникальных именованных сущностей из '{NAMED_ENTITIES_FILE}'.")
except FileNotFoundError:
    print(f"Предупреждение: Файл '{NAMED_ENTITIES_FILE}' не найден. Проверка на именованные сущности будет отключена.")
except Exception as e:
    print(f"Ошибка при чтении файла '{NAMED_ENTITIES_FILE}': {e}")
# ================================================

# --- Имена файлов ---
input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt'
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final.txt'

# --- Вспомогательные функции ---

def similarity(a, b):
    """Схожесть строк в % (без учета регистра)."""
    if not a or not b: return 0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100

def levenshtein_distance(s1, s2):
    """Вычисляет расстояние Левенштейна между s1 и s2 (case-insensitive)."""
    s1 = s1.lower()
    s2 = s2.lower()
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def load_prob_final_lemmas(filename):
    """Загружает леммы из prob_final_file в словарь {слово_lower: лемма}."""
    start_time = time.time()
    print(f"Загрузка лемм из '{filename}'...")
    lemma_map = {}
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    lemma = match.group(2).strip()
                    if word and lemma:
                        lemma_map[word.lower()] = lemma # Ключ всегда lowercase
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} лемм из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск в нем будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """Загружает 'Слово:' из photo_problems.txt в список."""
    start_time = time.time()
    print(f"Загрузка проблемных слов из '{filename}'...")
    word_list = []
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    if word:
                        word_list.append(word)
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} слов из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск схожих слов будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """
    Ищет первое слово в word_list (длина > 3, схожесть >= threshold).
    Приоритет словам со строчной буквы. Сравнение без учета регистра.
    """
    if not target_word: return None
    best_match_lower = None
    best_match_upper = None
    for word in word_list:
        if len(word) > 3:
            sim_score = similarity(target_word, word) # Сравнение уже без регистра
            if sim_score >= threshold:
                if not word[0].isalpha() or word[0].islower():
                    if best_match_lower is None:
                        best_match_lower = word
                        return best_match_lower
                elif best_match_upper is None:
                     best_match_upper = word
    return best_match_upper

# --- ОБНОВЛЕННАЯ Функция apply_fallback_logic ---
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
     """
     Определяет итоговую форму слова.
     Иерархия: именованная сущность (точно или близко) -> prob_final -> problems -> token -> word.
     """
     if not word:
         return token or ""

     # === Шаг 0: Проверка на именованную сущность (точное совпадение) ===
     word_lower = word.lower()
     if word_lower in named_entities:
         # Возвращаем сущность в оригинальном регистре из словаря
         # print(f"  [DEBUG] Шаг 0a: Слово '{word}' найдено точно в named_entities.")
         return named_entities_original_case.get(word_lower, word) # Запасной вариант - исходное слово

     # === Шаг 0b: Проверка на именованную сущность (близкое совпадение, <= 2) ===
     MAX_ENTITY_DISTANCE = 1
     # Оптимизация: проверяем только если слово не слишком короткое/длинное
     min_len_check = len(word) - MAX_ENTITY_DISTANCE
     max_len_check = len(word) + MAX_ENTITY_DISTANCE
     closest_entity_match = None
     min_dist = MAX_ENTITY_DISTANCE + 1 # Начинаем с расстояния больше максимального

     for entity_lower in named_entities:
         # Пропускаем сущности сильно разной длины
         if not (min_len_check <= len(entity_lower) <= max_len_check):
             continue

         dist = levenshtein_distance(word_lower, entity_lower) # Считаем расстояние (case-insensitive)
         if dist < min_dist:
             min_dist = dist
             closest_entity_match = entity_lower # Запоминаем lowercase сущность
             if min_dist == 0: # Если нашли точное совпадение в цикле, выходим
                 break
         # Если расстояние уже 0 или 1, нет смысла искать дальше меньшее
         if min_dist <= 1:
             break

     # Если нашли близкое совпадение (расстояние <= MAX_ENTITY_DISTANCE)
     if min_dist <= MAX_ENTITY_DISTANCE and closest_entity_match:
         # Возвращаем сущность в оригинальном регистре из словаря
         # print(f"  [DEBUG] Шаг 0b: Слово '{word}' близко к сущности '{closest_entity_match}' (dist={min_dist}).")
         return named_entities_original_case.get(closest_entity_match, word) # Запасной вариант - исходное слово
     # ======================================================================

     lemma_to_add = None # Переменная для результата шагов 2-5

     # --- Шаг 2: Поиск в prob_final_lemmas ---
     search_key_prob = word_lower # Уже есть lowercase версия
     found_lemma_in_prob = prob_lemmas.get(search_key_prob)

     if found_lemma_in_prob:
         # Правило 2: Если исходное 'Слово:' (word) было с большой буквы, используем его
         if word and word[0].isupper():
             lemma_to_add = word
         else:
             lemma_to_add = found_lemma_in_prob
         # Нашли на шаге 2, выходим из функции
         # print(f"  [DEBUG] Шаг 2: Найдено в prob_final. Итог: '{lemma_to_add}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 3: Поиск схожего в clustered_problem_words ---
     # Правило 1: Определяем цель поиска для similarity
     search_target_problems = word_lower if is_first_word else word
     found_similar = find_similar_word(search_target_problems, problem_words, 70)

     if found_similar:
         lemma_to_add = found_similar
         # Нашли на шаге 3, выходим из функции
         # print(f"  [DEBUG] Шаг 3: Найдено похожее '{found_similar}' для '{search_target_problems}'. Итог: '{lemma_to_add}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 4: Fallback на Токен ---
     if token:
         lemma_to_add = token
         # Нашли на шаге 4, выходим из функции
         # print(f"  [DEBUG] Шаг 4: Использован токен '{token}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 5: Самый крайний fallback на исходное Слово ---
     lemma_to_add = word
     # print(f"  [DEBUG] Шаг 5: Использовано исходное слово '{word}'")
     return lemma_to_add
# --- Конец ОБНОВЛЕННОЙ функции ---


# --- Основная логика ---

# 1. Проверка файлов и загрузка
if not os.path.exists(input_main_file):
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)

print("--- Загрузка вспомогательных данных ---")
overall_start_time = time.time()
# --- Загрузка вспомогательных данных происходит ВЫШЕ, перед функциями ---
print("--- Загрузка завершена ---")

# 2. Обработка основного файла
results = []
current_sentence_text = None
current_lemmas = []
current_word = None
current_token = None
is_first_word_in_sentence = False
processed_lines_count = 0
processed_sentences_count = 0
last_print_time = time.time()
line_num = 0

print(f"--- Обработка основного файла '{input_main_file}' ---")
try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line_num, line in enumerate(infile, 1):
            processed_lines_count += 1
            line = line.strip()

            current_time = time.time()
            if current_time - last_print_time > 5:
                print(f"Обработано строк: {processed_lines_count}, найдено предложений: {processed_sentences_count}...")
                last_print_time = current_time

            if not line: continue

            # --- Начало нового предложения ---
            if line.startswith("Предложение:"):
                if current_sentence_text is not None:
                    if current_word: # Завершаем последнее слово
                        final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                        if final_lemma: current_lemmas.append(final_lemma)
                    if current_lemmas: # Сохраняем результат
                        results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
                        processed_sentences_count += 1

                # Начинаем новое
                try: current_sentence_text = line.split(": ", 1)[1]
                except IndexError:
                    print(f"Предупреждение (строка {line_num}): Не удалось извлечь текст предложения: {line}")
                    current_sentence_text = ""
                current_lemmas = []
                current_word = None
                current_token = None
                is_first_word_in_sentence = True

            # --- Новое слово ---
            elif line.startswith("Слово:"):
                 if current_word: # Завершаем предыдущее слово
                     final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                     if final_lemma: current_lemmas.append(final_lemma)
                     if is_first_word_in_sentence: is_first_word_in_sentence = False # Сбрасываем флаг
                 # Запоминаем новое слово
                 try: current_word = line.split(": ", 1)[1]
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь слово: {line}")
                     current_word = None
                 current_token = None

            # --- Токен ---
            elif line.startswith("Токен:") and current_word:
                 try:
                     current_token = line.split(": ", 1)[1]
                     if not current_token: current_token = current_word
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь токен: {line}")
                     current_token = current_word

            # --- Лемма ---
            elif line.startswith("Лемма:") and current_word:
                lemma_from_file = ""
                try:
                    if ": " in line:
                        lemma_part = line.split(": ", 1)[1]
                        lemma_from_file = lemma_part.split('/', 1)[0].strip()
                    else: lemma_from_file = ""
                except IndexError: lemma_from_file = ""

                lemma_to_add = None
                # Шаг 1: Лемма из файла
                if lemma_from_file and len(lemma_from_file) >= 4:
                    lemma_to_add = lemma_from_file
                else:
                    # Шаги 0, 2-5
                    lemma_to_add = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)

                if lemma_to_add: current_lemmas.append(lemma_to_add)

                if is_first_word_in_sentence: is_first_word_in_sentence = False # Сбрасываем флаг
                current_word = None # Сбрасываем состояние слова
                current_token = None

    # --- Обработка последнего предложения ---
    if current_sentence_text is not None:
        if current_word: # Завершаем самое последнее слово
            final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
            if final_lemma: current_lemmas.append(final_lemma)
        if current_lemmas: # Сохраняем результат
             results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
             processed_sentences_count += 1

except FileNotFoundError:
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)
except Exception as e:
    print(f"Критическая ошибка при обработке основного файла (около строки {line_num}): {e}")
    traceback.print_exc()
    exit(1)

processing_time = time.time() - overall_start_time
print(f"--- Обработка основного файла завершена за {processing_time:.2f} сек. Обработано строк: {processed_lines_count} ---")

# 4. Запись результатов
print(f"--- Запись результатов в '{output_final_sentences_file}' ---")
write_start_time = time.time()
if results:
    try:
        with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
            for item in results:
                lemma_string = ' '.join(map(str, item['lemmas']))
                outfile.write(lemma_string.strip() + '\n')
        write_time = time.time() - write_start_time
        print(f"Запись завершена за {write_time:.2f} сек. Сохранено предложений: {len(results)}")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_final_sentences_file}': {e}")
else:
    print("Нет данных для записи в выходной файл.")

total_time = time.time() - overall_start_time
print(f"--- Скрипт завершил работу за {total_time:.2f} сек. ---")


Загружено 987 уникальных именованных сущностей из 'named_entities.txt'.
--- Загрузка вспомогательных данных ---
--- Загрузка завершена ---
--- Обработка основного файла 'lemmatized_not_punct_all.txt' ---
Обработано строк: 3775, найдено предложений: 43...
Обработано строк: 7505, найдено предложений: 86...
Обработано строк: 11193, найдено предложений: 130...
Обработано строк: 14573, найдено предложений: 172...
Обработано строк: 17773, найдено предложений: 210...
Обработано строк: 21547, найдено предложений: 253...
Обработано строк: 25359, найдено предложений: 301...
Обработано строк: 29389, найдено предложений: 350...
Обработано строк: 32817, найдено предложений: 390...
Обработано строк: 36585, найдено предложений: 436...
Обработано строк: 40377, найдено предложений: 480...
Обработано строк: 45121, найдено предложений: 548...
Обработано строк: 49123, найдено предложений: 601...
Обработано строк: 52821, найдено предложений: 646...
Обработано строк: 56751, найдено предложений: 699...
Обраб

In [ ]:
код ниже с удалением именов суш

In [7]:
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# ========== ЗАГРУЗКА ИМЕННЫХ СУЩНОСТЕЙ ==========
# Имя файла с именованными сущностями (одно слово/сущность на строку)
NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()
try:
    # Проверяем, существует ли файл перед открытием
    if not os.path.exists(NAMED_ENTITIES_FILE):
         print(f"Предупреждение: Файл '{NAMED_ENTITIES_FILE}' не найден. Проверка на именованные сущности будет отключена.")
    else:
        with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                entity = line.strip()
                if entity:
                    named_entities.add(entity.lower())  # Сохраняем в нижнем регистре для сравнения
        print(f"Загружено {len(named_entities)} уникальных именованных сущностей из '{NAMED_ENTITIES_FILE}'.")
except Exception as e:
    print(f"Ошибка при чтении файла '{NAMED_ENTITIES_FILE}': {e}")
# ================================================

# --- Имена файлов ---
input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt'
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final_new.txt'

# --- Вспомогательные функции ---

def similarity(a, b):
    """Схожесть строк в % (без учета регистра)."""
    if not a or not b: return 0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100

def load_prob_final_lemmas(filename):
    """Загружает леммы из prob_final_file в словарь {слово_lower: лемма}."""
    start_time = time.time()
    print(f"Загрузка лемм из '{filename}'...")
    lemma_map = {}
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)")
    loaded_count = 0
    try:
        # Проверяем существование файла
        if not os.path.exists(filename):
             print(f"Предупреждение: Файл '{filename}' не найден. Поиск в нем будет пропущен.")
             return lemma_map # Возвращаем пустой словарь

        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    lemma = match.group(2).strip()
                    if word and lemma:
                        lemma_map[word.lower()] = lemma # Ключ всегда lowercase
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} лемм из '{filename}' за {elapsed_time:.2f} сек.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """Загружает 'Слово:' из photo_problems.txt в список."""
    start_time = time.time()
    print(f"Загрузка проблемных слов из '{filename}'...")
    word_list = []
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,")
    loaded_count = 0
    try:
        # Проверяем существование файла
        if not os.path.exists(filename):
            print(f"Предупреждение: Файл '{filename}' не найден. Поиск схожих слов будет пропущен.")
            return word_list # Возвращаем пустой список

        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    if word:
                        word_list.append(word)
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} слов из '{filename}' за {elapsed_time:.2f} сек.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """
    Ищет первое слово в word_list (длина > 3, схожесть >= threshold).
    Приоритет словам со строчной буквы. Сравнение без учета регистра.
    """
    if not target_word: return None
    best_match_lower = None
    best_match_upper = None
    for word in word_list:
        if len(word) > 3:
            sim_score = similarity(target_word, word) # Сравнение уже без регистра
            if sim_score >= threshold:
                # Проверяем регистр кандидата из списка
                # Добавляем проверку word[0].isalpha() на случай неалфавитных символов
                if not word or not word[0].isalpha() or word[0].islower():
                    if best_match_lower is None:
                        best_match_lower = word
                        return best_match_lower # Нашли лучший (строчный), выходим
                elif best_match_upper is None:
                     best_match_upper = word # Запоминаем первый прописной
    return best_match_upper # Возвращаем прописной, если строчный не найден

# --- ИЗМЕНЕННАЯ Функция apply_fallback_logic ---
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
     """
     Определяет итоговую форму слова, если лемма из основного файла не подходит.
     Применяет иерархию: именованная сущность -> prob_final -> problems -> token -> word.
     ИМЕНОВАННЫЕ СУЩНОСТИ С БОЛЬШОЙ БУКВЫ УДАЛЯЮТСЯ (ВОЗВРАЩАЕТСЯ None).
     """
     if not word:
         return token or "" # Возвращаем токен или пустую строку, если слова нет

     # === Шаг 0: Проверка на именованную сущность ===
     if word.lower() in named_entities:
         # Проверяем регистр ОРИГИНАЛЬНОГО слова
         # Добавлена проверка word[0].isalpha() на случай, если слово начинается не с буквы
         if word and word[0].isalpha() and word[0].isupper():
             # Если слово с большой буквы и это именованная сущность - УДАЛЯЕМ (возвращаем None)
             # print(f"  [DEBUG] Шаг 0: Сущность '{word}' с большой буквы. Удаляем.")
             return None # Сигнал для удаления слова из предложения
         else:
             # Если слово с маленькой буквы (или не с буквы) и это именованная сущность - ОСТАВЛЯЕМ КАК ЕСТЬ
             # print(f"  [DEBUG] Шаг 0: Сущность '{word}' с маленькой буквы. Оставляем.")
             return word
     # ==============================================

     lemma_to_add = None # Переменная для результата шагов 2-5

     # --- Шаг 2: Поиск в prob_final_lemmas ---
     search_key_prob = word.lower()
     found_lemma_in_prob = prob_lemmas.get(search_key_prob)

     if found_lemma_in_prob:
         # Правило 2: Если исходное 'Слово:' (word) было с большой буквы, используем его
         # (Это правило теперь менее вероятно сработает для сущностей,
         # так как они обрабатываются в Шаге 0)
         if word and word[0].isalpha() and word[0].isupper():
             lemma_to_add = word
         else:
             lemma_to_add = found_lemma_in_prob
         return lemma_to_add

     # --- Шаг 3: Поиск схожего в clustered_problem_words ---
     search_target_problems = word.lower() if is_first_word else word
     found_similar = find_similar_word(search_target_problems, problem_words, 70)

     if found_similar:
         return found_similar # Нашли похожее, возвращаем его

     # --- Шаг 4: Fallback на Токен ---
     if token:
         return token # Используем токен

     # --- Шаг 5: Самый крайний fallback на исходное Слово ---
     return word # Если ничего не нашли, возвращаем исходное слово
# --- Конец ИЗМЕНЕННОЙ функции ---


# --- Основная логика ---

# 1. Проверка основного входного файла
if not os.path.exists(input_main_file):
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)

print("--- Загрузка вспомогательных данных ---")
overall_start_time = time.time()
# Загрузка данных
prob_final_lemmas = load_prob_final_lemmas(prob_final_file)
clustered_problem_words = load_clustered_problem_words(problems_clustered_file)
# named_entities загружены в самом начале
print("--- Загрузка завершена ---")

# 2. Обработка основного файла
results = []
current_sentence_text = None
current_lemmas = []
current_word = None
current_token = None
is_first_word_in_sentence = False # Флаг первого слова
processed_lines_count = 0
processed_sentences_count = 0
last_print_time = time.time()
line_num = 0 # Инициализация перед циклом

print(f"--- Обработка основного файла '{input_main_file}' ---")
try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line_num, line in enumerate(infile, 1):
            processed_lines_count += 1
            line = line.strip()

            current_time = time.time()
            if current_time - last_print_time > 5: # Индикатор прогресса
                print(f"Обработано строк: {processed_lines_count}, найдено предложений: {processed_sentences_count}...")
                last_print_time = current_time

            if not line: continue # Пропускаем пустые строки

            # --- Начало нового предложения ---
            if line.startswith("Предложение:"):
                if current_sentence_text is not None:
                    # Завершаем последнее слово пред. предложения
                    if current_word:
                        final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                        # Добавляем слово, только если оно не было удалено (не None)
                        if final_lemma is not None:
                            current_lemmas.append(final_lemma)
                    # Сохраняем результат, если есть леммы
                    if current_lemmas:
                        results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
                        processed_sentences_count += 1

                # Начинаем новое предложение
                try:
                    current_sentence_text = line.split(": ", 1)[1]
                except IndexError:
                    print(f"Предупреждение (строка {line_num}): Не удалось извлечь текст предложения: {line}")
                    current_sentence_text = ""
                current_lemmas = []
                current_word = None
                current_token = None
                is_first_word_in_sentence = True # Устанавливаем флаг для нового предложения

            # --- Новое слово ---
            elif line.startswith("Слово:"):
                 # Завершаем предыдущее слово, если оно не было обработано строкой "Лемма:"
                 if current_word:
                     final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                     # Добавляем слово, только если оно не было удалено (не None)
                     if final_lemma is not None:
                         current_lemmas.append(final_lemma)
                     # Сбрасываем флаг после обработки, если это было первое слово
                     if is_first_word_in_sentence:
                         is_first_word_in_sentence = False

                 # Запоминаем новое слово
                 try:
                     current_word = line.split(": ", 1)[1]
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь слово из: {line}")
                     current_word = None
                 current_token = None # Сбрасываем токен для нового слова

            # --- Токен ---
            elif line.startswith("Токен:") and current_word:
                 try:
                     current_token = line.split(": ", 1)[1]
                     if not current_token: current_token = current_word # Если токен пуст
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь токен из: {line}")
                     current_token = current_word

            # --- Лемма ---
            elif line.startswith("Лемма:") and current_word:
                lemma_from_file = ""
                try:
                    if ": " in line:
                        lemma_part = line.split(": ", 1)[1]
                        # Берем первую лемму до '/'
                        lemma_from_file = lemma_part.split('/', 1)[0].strip()
                    else: lemma_from_file = "" # Если нет ": "
                except IndexError: lemma_from_file = "" # Если не удалось разделить по ": "

                # --- Применяем иерархию ---
                lemma_to_add = None
                # Шаг 1: Лемма из файла (если >= 4 символов)
                if lemma_from_file and len(lemma_from_file) >= 4:
                    lemma_to_add = lemma_from_file
                    # print(f"  [DEBUG] Шаг 1: Использована лемма из файла: '{lemma_to_add}' для слова '{current_word}'")
                else:
                    # --- Шаг 0 и Шаги 2-5 ---
                    # Функция сама выполнит проверку на named_entity (Шаг 0)
                    lemma_to_add = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)

                # Добавляем слово, только если оно не было удалено (не None)
                if lemma_to_add is not None:
                    current_lemmas.append(lemma_to_add)

                # Сбрасываем флаг первого слова
                if is_first_word_in_sentence:
                    is_first_word_in_sentence = False

                # Сбрасываем состояние слова, т.к. оно обработано
                current_word = None
                current_token = None

    # --- Обработка последнего предложения ---
    if current_sentence_text is not None:
        if current_word: # Завершаем самое последнее слово
            final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
            # Добавляем слово, только если оно не было удалено (не None)
            if final_lemma is not None:
                current_lemmas.append(final_lemma)
        # Сохраняем результат, если есть леммы
        if current_lemmas:
             results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
             processed_sentences_count += 1

except FileNotFoundError: # Эта проверка дублируется, но оставим для надежности
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)
except Exception as e:
    print(f"Критическая ошибка при обработке основного файла (около строки {line_num}): {e}")
    traceback.print_exc()
    exit(1)

processing_time = time.time() - overall_start_time
print(f"--- Обработка основного файла завершена за {processing_time:.2f} сек. Обработано строк: {processed_lines_count} ---")

# 4. Запись результатов
print(f"--- Запись результатов в '{output_final_sentences_file}' ---")
write_start_time = time.time()
if results:
    try:
        with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
            for item in results:
                lemma_string = ' '.join(map(str, item['lemmas']))
                outfile.write(lemma_string.strip() + '\n')
        write_time = time.time() - write_start_time
        print(f"Запись завершена за {write_time:.2f} сек. Сохранено предложений: {len(results)}")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_final_sentences_file}': {e}")
else:
    print("Нет данных для записи в выходной файл.")

total_time = time.time() - overall_start_time
print(f"--- Скрипт завершил работу за {total_time:.2f} сек. ---")


Загружено 992 уникальных именованных сущностей из 'named_entities.txt'.
--- Загрузка вспомогательных данных ---
Загрузка лемм из 'prob_lemmatized_photo_all_final.txt'...
Загружено 7265 лемм из 'prob_lemmatized_photo_all_final.txt' за 0.01 сек.
Загрузка проблемных слов из 'photo_problems.txt'...
Загружено 2566 слов из 'photo_problems.txt' за 0.00 сек.
--- Загрузка завершена ---
--- Обработка основного файла 'lemmatized_not_punct_all.txt' ---
Обработано строк: 4477, найдено предложений: 50...
Обработано строк: 8703, найдено предложений: 101...
Обработано строк: 12659, найдено предложений: 151...
Обработано строк: 16285, найдено предложений: 192...
Обработано строк: 20501, найдено предложений: 240...
Обработано строк: 25171, найдено предложений: 299...
Обработано строк: 29677, найдено предложений: 354...
Обработано строк: 33457, найдено предложений: 398...
Обработано строк: 37197, найдено предложений: 442...
Обработано строк: 41713, найдено предложений: 500...
Обработано строк: 46405, най

In [ ]:
# почищенный от комментов
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# Именов сущ 
NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()

try:
    with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            entity = line.strip()
            if entity:
                named_entities.add(entity.lower())  # Нормализация для сравнения
        print(f"Загружено {len(named_entities)} уникальных сущностей.")
except Exception as e:
    print(f"Ошибка загрузки сущностей: {e}")


input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt' 
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final_new.txt'


def similarity(a, b):
    """вычисление процентной схожести строк"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100 if a and b else 0

def load_prob_final_lemmas(filename):
    """ парсинг файла с предобработанными леммами"""
    lemma_map = {}
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)", line)
                if match:
                    lemma_map[match.group(1).lower()] = match.group(2).strip()
    except Exception as e:
        print(f"Ошибка чтения лемм: {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """ извлечение проблемных слов для fuzzywuzzy-поиска"""
    word_list = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,", line)
                if match:
                    word_list.append(match.group(1))
    except Exception as e:
        print(f"Ошибка чтения проблемных слов: {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """ поиск приблизительного совпадения с приоритетом строчных вариантов"""
    best_match = None
    for word in word_list:
        if len(word) > 3 and similarity(target_word, word) >= threshold:
            # Предпочтение к нижнему регистру
            if not best_match or word[0].islower():
                best_match = word
                if word[0].islower():
                    break  # Лучший вар найден
    return best_match

# Процесс
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
    """иерархический выбор оптимальной леммы"""
    if not word:
        return token or ""  # защита от пустых значений

    #  1: Именованные сущности
    if word.lower() in named_entities:
        # Удаление сущностей с заглавной буквы
        return None if word[0].isupper() else word
    
    #  2: Предобработанные леммы
    lemma = prob_lemmas.get(word.lower())
    if lemma:
        return word if word[0].isupper() else lemma
    
    # 3: Нечеткий поиск в проблемных словах
    similar = find_similar_word(word.lower() if is_first_word else word, problem_words)
    if similar:
        return similar
    
    # 4-5: Токен без префикса или исходное слово
    return token or word

# ОСНОВА
print(" Инициализация обработки ")
start_time = time.time()

prob_final_lemmas = load_prob_final_lemmas(prob_final_file)
clustered_problem_words = load_clustered_problem_words(problems_clustered_file)

results = []
current_context = {
    "text": None,
    "lemmas": [],
    "current_word": None,
    "current_token": None,
    "is_first_word": False
}

try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            # Обработка предложений
            if line.startswith("Предложение:"):
                if current_context["text"] and current_context["lemmas"]:
                    results.append({
                        "sentence": current_context["text"], 
                        "lemmas": current_context["lemmas"]
                    })
                current_context = {
                    "text": line.split(": ", 1)[1],
                    "lemmas": [],
                    "current_word": None,
                    "current_token": None,
                    "is_first_word": True
                }

            # Обработка слов
            elif line.startswith("Слово:"):
                if current_context["current_word"]:
                    lemma = apply_fallback_logic(
                        current_context["current_word"],
                        current_context["current_token"],
                        prob_final_lemmas,
                        clustered_problem_words,
                        current_context["is_first_word"]
                    )
                    if lemma is not None:
                        current_context["lemmas"].append(lemma)
                current_context["current_word"] = line.split(": ", 1)[1]
                current_context["current_token"] = None

            # Обработка токенов
            elif line.startswith("Токен:") and current_context["current_word"]:
                current_context["current_token"] = line.split(": ", 1)[1] or current_context["current_word"]

            # Финализация леммы
            elif line.startswith("Лемма:") and current_context["current_word"]:
                lemma = line.split(": ", 1)[1].split('/', 1)[0].strip()
                final_lemma = lemma if len(lemma) >= 4 else apply_fallback_logic(
                    current_context["current_word"],
                    current_context["current_token"],
                    prob_final_lemmas,
                    clustered_problem_words,
                    current_context["is_first_word"]
                )
                if final_lemma is not None:
                    current_context["lemmas"].append(final_lemma)
                current_context["current_word"] = None

    # Финализация последнего предложения
    if current_context["text"] and current_context["lemmas"]:
        results.append({
            "sentence": current_context["text"], 
            "lemmas": current_context["lemmas"]
        })

except Exception as e:
    print(f"Критическая ошибка: {str(e)}")
    traceback.print_exc()
    exit(1)

# Резы
try:
    with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
        for item in results:
            outfile.write(' '.join(item['lemmas']) + '\n')
    print(f"Успешно сохранено {len(results)} предложений.")
except Exception as e:
    print(f"Ошибка сохранения: {str(e)}")
    traceback.print_exc()

print(f"Общее время выполнения: {time.time()-start_time:.2f} сек.")

#### **8. Конечная очистка и подсчёт слов**

In [11]:
import re
import os
import traceback # Для вывода подробной информации об ошибках

# --- Конфигурация ---
INPUT_FILE = "output_lemmas_sentences_final.txt" # Исходный файл с предложениями после лемматизации
OUTPUT_FILE = "corrected_sentences_manual_v1.txt" # Файл для исправленных предложений (версия 1)

# Ограничение для тестирования: сколько строк обработать (-1 для обработки всего файла)
PROCESS_LIMIT = -1 # Установите, например, 100 или 1000 для быстрой проверки

# --- Правила Исправления ---
def apply_correction_rules(word):
    """
    Применяет набор правил ручной коррекции к одному слову.
    Добавляйте новые правила в эту функцию.
    """
    corrected = word
# -------правки
    if corrected.startswith("db"): # Простая проверка для начала
         # Проверим, что после db есть еще символы, чтобы не превратить "db" в "b"
         if len(corrected) > 2:
              corrected = "b" + corrected[2:]


    return corrected

# --- Основной процесс ---
def main():
    print("--- Начало ручной правки текста ---")
    print(f"Чтение из: {INPUT_FILE}")
    print(f"Запись в: {OUTPUT_FILE}")
    if PROCESS_LIMIT > 0:
        print(f"Ограничение обработки: {PROCESS_LIMIT} строк")

    # Проверка наличия входного файла
    if not os.path.exists(INPUT_FILE):
        print(f"КРИТИЧЕСКАЯ ОШИБКА: Входной файл '{INPUT_FILE}' не найден.")
        return # Выход из функции main

    processed_lines = 0
    written_lines = 0
    line_num = 0 # Для отслеживания строки при ошибке

    try:
        # Открываем оба файла одновременно
        with open(INPUT_FILE, 'r', encoding='utf-8') as infile, \
             open(OUTPUT_FILE, 'w', encoding='utf-8') as outfile:

            # Читаем входной файл построчно
            for line_num, line in enumerate(infile, 1):
                processed_lines += 1
                original_sentence = line.strip()

                # Если строка пустая, просто пишем перенос строки в выходной файл
                if not original_sentence:
                    outfile.write('\n')
                    written_lines += 1
                    continue

                # Делим предложение на слова по пробелам
                words = original_sentence.split()
                corrected_words = []

                # Применяем правила к каждому слову
                for word in words:
                    corrected_word = apply_correction_rules(word)
                    corrected_words.append(corrected_word)

                # Собираем исправленное предложение обратно
                corrected_sentence = " ".join(corrected_words)

                # Записываем исправленное предложение в выходной файл
                outfile.write(corrected_sentence + '\n')
                written_lines += 1

                # Индикатор прогресса (вывод каждые 5000 строк)
                if processed_lines % 5000 == 0:
                    print(f"Обработано строк: {processed_lines}...")

                # Проверка лимита для тестирования
                if PROCESS_LIMIT > 0 and processed_lines >= PROCESS_LIMIT:
                    print(f"Достигнуто ограничение обработки в {PROCESS_LIMIT} строк.")
                    break

        print("--- Обработка завершена ---")
        print(f"Всего обработано строк из входного файла: {processed_lines}")
        print(f"Записано строк в выходной файл: {written_lines}")

    except Exception as e:
        # Вывод подробной информации в случае непредвиденной ошибки
        print(f"\n!!! Произошла ошибка во время обработки (около строки {line_num}) !!!")
        print(f"Тип ошибки: {type(e).__name__}")
        print(f"Сообщение: {e}")
        print("--- Трассировка стека ---")
        traceback.print_exc()
        print("-------------------------")
        print("Запись в выходной файл может быть неполной или некорректной.")

# Запуск основной функции, если скрипт выполняется напрямую
if __name__ == "__main__":
    main()


--- Начало ручной правки текста ---
Чтение из: output_lemmas_sentences_final.txt
Запись в: corrected_sentences_manual_v1.txt
--- Обработка завершена ---
Всего обработано строк из входного файла: 3649
Записано строк в выходной файл: 3649


In [8]:
import re
import os
import string
import traceback 

def filter_lemmas_and_sentences(input_file, output_file):
    filtered_lines = []
    if not os.path.exists(input_file):
        print(f"Ошибка: Входной файл '{input_file}' не найден.")
        return


    VOWELS_LOWER = set('aeiou')
    LATIN_CONSONANTS_LOWER = set(string.ascii_lowercase) - VOWELS_LOWER

    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line_num, line in enumerate(infile, 1): 
                words = line.strip().split()
                processed_words = []

                for word in words:
                    current_word = word.split('/', 1)[0].strip() if '/' in word else word

                    if not current_word: continue 
                    current_word_lower = current_word.lower()

                    word_to_check = current_word_lower 
                    prefix_removed = False 

                    if len(current_word_lower) >= 3 and current_word_lower.startswith(('d', 'l')):
                        char1 = current_word_lower[1]
                        char2 = current_word_lower[2]
                        if char1 in LATIN_CONSONANTS_LOWER and char2 in LATIN_CONSONANTS_LOWER:
                            current_word_lower = current_word_lower[1:] # префикс
                            prefix_removed = True

                    if len(current_word_lower) >= 3:
                        processed_words.append(current_word_lower)


                # количество слов в предложении 
                if len(processed_words) > 2:
                    filtered_lines.append(" ".join(processed_words) + "\n")

    except Exception as e:
        print(f"Ошибка при чтении файла '{input_file}' (строка ~{line_num}): {e}")
        traceback.print_exc() # Добавим вывод стека ошибок
        return

    # рез в файле
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.writelines(filtered_lines)
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_file}': {e}")
        traceback.print_exc()


if __name__ == "__main__":
    input_file = 'output_lemmas_sentences_final_new.txt'
    output_file = 'photo_sentences_filtered_new.txt'

    filter_lemmas_and_sentences(input_file, output_file)

    if os.path.exists(output_file):
         if os.path.getsize(output_file) > 0:
              print(f"Результаты записаны в {output_file}")
         else:
              print(f"Выходной файл {output_file} создан, но он пуст (нет подходящих строк).")
    else:
         print(f"Выходной файл {output_file} не был создан (возможно, из-за ошибки).")

Результаты записаны в photo_sentences_filtered_new.txt


In [2]:
import re

def count_unique_words(input_file):
    """
    Читает файл и считает количество уникальных слов.
    """
    unique_words = set()
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            words = line.strip().split()
            for word in words:
                unique_words.add(word)
    
    return len(unique_words)


if __name__ == "__main__":
    input_file = 'photo_sentences_filtered_new.txt'
    unique_word_count = count_unique_words(input_file)
    print(f"Количество уникальных слов в файле {input_file}: {unique_word_count}")

Количество уникальных слов в файле photo_sentences_filtered_new.txt: 4719


__________________________________________________________________________________
слов уникальных - 4719 

предложений - 3445
_____________________________________________________________________

#### **9. Объединение художественных и газетных текстов**

In [46]:
# Объединение предложений из двух файлов в один

input_file1 = "photo_sentences_filtered_new.txt"
input_file2 = "output_lemmas_sentences_filtered.txt"
output_file = "general_sentences_filtered.txt"

def merge_sentence_files(file1, file2, outfile):
    sentences = []
    for fname in [file1, file2]:
        try:
            with open(fname, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line:
                        sentences.append(line)
        except Exception as e:
            print(f"Ошибка при чтении файла {fname}: {e}")
    try:
        with open(outfile, "w", encoding="utf-8") as out:
            for sent in sentences:
                out.write(sent + "\n")
        print(f"Объединено {len(sentences)} предложений. Результат записан в {outfile}")
    except Exception as e:
        print(f"Ошибка при записи в файл {outfile}: {e}")

if __name__ == "__main__":
    merge_sentence_files(input_file1, input_file2, output_file)


Объединено 14165 предложений. Результат записан в general_sentences_filtered.txt


In [3]:
import re
import os
import string # Импортируем модуль для работы со знаками пунктуации
import traceback # Для вывода подробной информации об ошибках

# --- Конфигурация ---
FILENAME = "named_entities.txt" # Имя файла для чтения и записи

def clean_named_entities_file(filepath):
    """
    Читает файл, удаляет из строк кириллические слова и знаки пунктуации,
    удаляет пустые строки и перезаписывает исходный файл.
    """
    print(f"--- Начало очистки файла: {filepath} ---")

    # 1. Проверка существования файла
    if not os.path.exists(filepath):
        print(f"ОШИБКА: Файл '{filepath}' не найден.")
        return # Выход из функции, если файла нет

    original_lines = []
    line_num = 0
    try:
        # 2. Чтение всех строк из файла
        print("Чтение исходного файла...")
        with open(filepath, 'r', encoding='utf-8') as infile:
            original_lines = infile.readlines()
        print(f"Прочитано строк: {len(original_lines)}")

    except Exception as e:
        print(f"ОШИБКА при чтении файла '{filepath}': {e}")
        traceback.print_exc()
        return # Выход при ошибке чтения

    cleaned_lines = []
    processed_count = 0
    removed_cyrillic_count = 0
    removed_punct_count = 0
    kept_lines_count = 0

    # 3. Обработка каждой строки
    print("Обработка строк...")
    cyrillic_word_pattern = re.compile(r'\b[а-яА-ЯёЁ]+\b') # Паттерн для кириллических слов
    # Создаем паттерн для знаков пунктуации (экранируем специальные символы)
    punctuation_pattern = re.compile(f'[{re.escape(string.punctuation)}]')

    for line_num, line in enumerate(original_lines, 1):
        processed_count += 1
        original_line_content = line.strip() # Сохраним для отладки, если нужно

        # Пропускаем изначально пустые строки
        if not original_line_content:
            continue

        # Удаляем кириллические слова
        line_no_cyrillic = cyrillic_word_pattern.sub('', line)
        if line_no_cyrillic != line:
            removed_cyrillic_count += 1

        # Удаляем знаки пунктуации
        line_no_punct = punctuation_pattern.sub('', line_no_cyrillic)
        if line_no_punct != line_no_cyrillic:
             removed_punct_count += 1 # Считаем строки, где была удалена пунктуация

        # Удаляем лишние пробелы и пробелы по краям
        cleaned_line = re.sub(r'\s+', ' ', line_no_punct).strip()

        # Добавляем строку в результат, только если она не пустая после очистки
        if cleaned_line:
            cleaned_lines.append(cleaned_line)
            kept_lines_count += 1
        # else: # Отладка: какие строки стали пустыми
        #    if original_line_content: # Если строка не была пустой изначально
        #        print(f"  Строка {line_num} стала пустой после очистки: '{original_line_content}'")

    print(f"Обработка строк завершена. Итого строк для записи: {kept_lines_count}")
    #print(f"Строк с удаленными кириллическими словами: {removed_cyrillic_count}")
    #print(f"Строк с удаленной пунктуацией: {removed_punct_count}")

    # 4. Перезапись файла очищенными строками
    try:
        print(f"Перезапись файла: {filepath}...")
        with open(filepath, 'w', encoding='utf-8') as outfile:
            for line in cleaned_lines:
                outfile.write(line + '\n') # Добавляем перенос строки
        print("Файл успешно перезаписан.")

    except Exception as e:
        print(f"ОШИБКА при записи в файл '{filepath}': {e}")
        traceback.print_exc()

# Запуск основной функции
if __name__ == "__main__":
    clean_named_entities_file(FILENAME)
    print("--- Скрипт завершил работу ---")



--- Начало очистки файла: named_entities.txt ---
Чтение исходного файла...
Прочитано строк: 1079
Обработка строк...
Обработка строк завершена. Итого строк для записи: 1079
Перезапись файла: named_entities.txt...
Файл успешно перезаписан.
--- Скрипт завершил работу ---


### **Модель, обучение**

In [6]:
import io

def load_corpus(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    documents = []
    for line in fin:
        documents.append(line.split())
    return documents

In [7]:
# Эта функция сохраняет словарь (где ключи - слова, а значения - их векторные представления) в файл.
# Это необходимо, чтобы сохранить результаты обучения модели.
def save_dictionary(fname, dictionary, args):
    length, dimension = args # длина словаря и размерность векторов
    fin = io.open(fname, 'w', encoding='utf-8')
    fin.write('%d %d\n' % (length, dimension))
    for word in dictionary:
        fin.write('%s %s\n' % (word, ' '.join(map(str, dictionary[word]))))

# Эта функция читает словарь из файла, созданного с помощью функции save_dictionary.
# Это позволяет загрузить ранее обученную модель.
def load_dictionary(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    length, dimension = map(int, fin.readline().split())
    dictionary = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        dictionary[tokens[0]] = map(float, tokens[1:])
    return dictionary

In [8]:
documents = load_corpus('general_sentences_filtered.txt')
documents

[['himdvəjə',
  'bdərkulutə',
  'imperialisti',
  'narazьjuta',
  'min',
  'huqma',
  'inь',
  'qul',
  'ojma',
  'qjəşə',
  'gərvusi',
  'dana',
  'xora',
  'tixxuв',
  'xaraja',
  'ijna',
  'burzuazija',
  'nәpilva',
  'kәdro',
  'tammam',
  'qiѕ',
  'alma',
  'tamamta',
  'ara',
  'alma',
  'midri',
  'kim',
  'malsomь',
  'dmillet',
  'muxə',
  'xarta',
  'bojkot',
  'matta',
  'inь',
  'јanb',
  'maşjatta',
  'alma',
  'huqma',
  'maşjatta',
  'ijdə'],
 ['inь', 'dьvjan', 'biltelgraр'],
 ['huqma',
  'alma',
  'palaţa',
  'plotva',
  'sәmo',
  'bivaјa',
  'gura',
  'mədjuni',
  'tund',
  'bati',
  'bidvoxila',
  'dnəşi'],
 ['qoma',
  'riş',
  'burzuazaji',
  'pjəşə',
  'dvijki',
  'komitet',
  'malsomь',
  'dmillet',
  'paşa',
  'muxə',
  'dərqul',
  'şarrat'],
 ['raba',
  'mədjuni',
  'billəxina',
  'bdərkulutə',
  'huqma',
  'inь',
  'pal',
  'biplasla',
  'alma',
  'səmə',
  'вevaja',
  'matta',
  'sudra',
  'smuqə',
  'garşələn',
  'hiççum',
  'mudsrelun',
  'tixxuв',
  'bati',


In [9]:
len(documents)

14165

In [10]:
dimension = 80

In [11]:
%%time
from gensim.models import Word2Vec

#dimension = 8 # размерность векторов слов (эмбеддингов).
#model = Word2Vec(sentences=documents, vector_size=dimension, min_count=1)    было 50 вектор-сайз

model2 = Word2Vec(
    sentences=documents,
    vector_size=dimension,       # размерность
    window=3,            # размер окна
    min_count=1,         # минимальная частота слова
    sg=0,              # CBOW
    negative=15,          # negative sampling
    alpha=0.025,         # learning rate
    min_alpha=0.0001,     # Минимал learning rate
    epochs=30
)


CPU times: user 32.4 s, sys: 733 ms, total: 33.1 s
Wall time: 12.5 s


'\nmodel = Word2Vec(\n    sentences=documents,\n    vector_size=dimension,       # Уменьшение размерности\n    window=7,            # Шире контекст\n    min_count=5,         # Игнорировать редкие слова\n    sg=1,                # Переход на skip-gram\n    negative=10,         # Умеренный negative sampling\n    alpha=0.03,         \n    min_alpşəxtha=0.0007,\n    epochs=25,           # Сокращение эпох\n    sample=1e-4,        # Субсэмплинг частых слов\n    workers=4           # Параллелизация\n)\n'

In [12]:
# создает словарь dictionary, где ключами являются слова, а значениями - их векторные представления.
dictionary = {key : model2.wv[key] for key in model2.wv.key_to_index}

In [13]:
len(dictionary)

9019

In [54]:
model2.wv.most_similar('çətun', topn = 100) 

[('ənənqəjə', 0.6578905582427979),
 ('вimxəju', 0.6351898312568665),
 ('tuxminnivin', 0.6302333474159241),
 ('itvəlij', 0.6165546774864197),
 ('вetalah', 0.6118181943893433),
 ('xoşu', 0.6077311038970947),
 ('məвsimənə', 0.6040380001068115),
 ('xdəjə', 0.6005393266677856),
 ('şupra', 0.5906239151954651),
 ('maprumь', 0.5865479111671448),
 ('degə', 0.5787451267242432),
 ('lazьm', 0.5760652422904968),
 ('dulgərutə', 0.5744701027870178),
 ('ţraxa', 0.5721561908721924),
 ('çançurь', 0.5698667764663696),
 ('maççьвana', 0.5694267749786377),
 ('aqьldar', 0.5693979859352112),
 ('vetə', 0.567062258720398),
 ('huçiççivəxvə', 0.5632379055023193),
 ('xijivin', 0.561469316482544),
 ('xşəxə', 0.5598565936088562),
 ('xoşij', 0.559091329574585),
 ('aldajatь', 0.5576958060264587),
 ('şijşə', 0.5553222894668579),
 ('purpьţţьna', 0.5545424818992615),
 ('məlupi', 0.5485862493515015),
 ('punilun', 0.5467336177825928),
 ('aççьzvalьj', 0.5449274778366089),
 ('tapavut', 0.5448868870735168),
 ('jaqrunь', 0.544

In [98]:
save_dictionary('urmi_dictionary_general.txt', dictionary, (len(dictionary), dimension))

In [100]:
loaded_dictionary = load_dictionary('urmi_dictionary_general.txt')
len(dictionary) == len(loaded_dictionary)

True

In [101]:
import os
import shutil
import traceback 

def remove_first_line(filepath):
    """
    Удаляет первую строку из указанного файла, тк там прописана размерность векторов
    Создает временный файл, копирует все строки, кроме первой, а затем заменяет исходный файл временным.
    """
    print(f"Попытка удалить первую строку из файла: {filepath}")

    if not os.path.exists(filepath):
        print(f"ОШИБКА: Файл '{filepath}' не найден.")
        return False

    # Создание имени для временного файла  .tmp 
    temp_filepath = filepath + ".tmp"

    try:
        with open(filepath, 'r', encoding='utf-8') as infile, \
             open(temp_filepath, 'w', encoding='utf-8') as outfile:
 
            first_line = infile.readline()

            if not first_line and os.path.getsize(filepath) == 0:
                 print(f"Предупреждение: Файл '{filepath}' пуст. Ничего не удалено.")
                 return True

            shutil.copyfileobj(infile, outfile)

        # замена исходного файла временным
        # проверка на кучу ошибок
        try:
            os.replace(temp_filepath, filepath)
            print(f"Первая строка успешно удалена из файла '{filepath}'.")
            return True
        except OSError as e:
            print(f"ОШИБКА при замене файла '{filepath}' временным файлом '{temp_filepath}': {e}")
            if os.path.exists(temp_filepath):
                try:
                    os.remove(temp_filepath)
                except OSError:
                    print(f"Не удалось удалить временный файл '{temp_filepath}' после ошибки замены.")
            return False

    except FileNotFoundError:
        print(f"ОШИБКА: Файл '{filepath}' не найден во время обработки.")
        return False
    except Exception as e:
        print(f"ОШИБКА при обработке файла '{filepath}': {e}")
        traceback.print_exc()
        if os.path.exists(temp_filepath):
            try:
                os.remove(temp_filepath)
                print(f"Временный файл '{temp_filepath}' удален после ошибки.")
            except OSError:
                 print(f"Не удалось удалить временный файл '{temp_filepath}' после ошибки.")
        return False


if __name__ == "__main__":
    TARGET_FILENAME = "urmi_dictionary_general.txt"

    if remove_first_line(TARGET_FILENAME):
        print("Операция завершена успешно.")
    else:
        print("Операция завершилась с ошибкой.")


Попытка удалить первую строку из файла: urmi_dictionary_general.txt
Первая строка успешно удалена из файла 'urmi_dictionary_general.txt'.
Операция завершена успешно.


#### **Автоматизированный перевод дырок**

In [3]:
import re
import os
import traceback

def load_translations(lexemes_filepath="lexemes.txt"):
    """
    Загружает переводы (trans_ru) из файла lexemes.txt в словарь.
    Ключи словаря - формы слов из поля 'lex:' в нижнем регистре.
    """
    translations = {}
    
    if not os.path.exists(lexemes_filepath):
        print(f"ОШИБКА: Файл '{lexemes_filepath}' не найден.")
        return translations 
        
    print(f"Загрузка переводов из файла: {lexemes_filepath}")

    # регулярки для извлечения
    lex_pattern = re.compile(r"^\s*lex:\s*(.+)$")
    trans_ru_pattern = re.compile(r"^\s*trans_ru:\s*(.+)$")

    current_lexemes = []
    current_trans_ru = None
    line_num = 0

    
    try:
        with open(lexemes_filepath, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line_stripped = line.strip()

                # начало записи
                if line_stripped.startswith("-lexeme"):
                    # сохранение прошлой записи, если она окей
                    if current_lexemes and current_trans_ru:
                        for lex_form in current_lexemes:
                            # сохранение ключа в нижнем регистре
                            translations[lex_form.lower()] = current_trans_ru
                    #сброс для новой записи
                    current_lexemes = []
                    current_trans_ru = None
                    continue 
                    
                #поиск поля 'lex:'
                lex_match = lex_pattern.match(line_stripped)
                if lex_match and not current_lexemes: # пока не нашли поиск
                    lex_value = lex_match.group(1).strip()
                    variations = [v.strip() for v in lex_value.split('/') if v.strip()]
                    current_lexemes.extend(variations)
                    continue

                #поиск поля 'trans_ru:'
                trans_ru_match = trans_ru_pattern.match(line_stripped)
                if trans_ru_match and not current_trans_ru: 
                    current_trans_ru = trans_ru_match.group(1).strip()
                    continue

            # сохраняем последнее
            if current_lexemes and current_trans_ru:
                for lex_form in current_lexemes:
                    translations[lex_form.lower()] = current_trans_ru

    except Exception as e:
        print(f"ОШИБКА при чтении/парсинге файла '{lexemes_filepath}' (строка ~{line_num}): {e}")
        traceback.print_exc()

    print(f"Загружено переводов для {len(translations)} уникальных форм слов.")
    return translations


def translate_lines_keep_original(input_text, translations_dict):
    """
    Принимает текст и словарь переводов.
    Заменяет слова (разделенные ' | ') на их переводы или оставляет слово как есть.
    Добавляет пустую строку между результатами.
    """
    output_lines = []
    lines = input_text.strip().split('\n') # Разбиваем на строки

    for line in lines:
        # Разбиение строки на слова по '|' и минус пробелы
        original_words = [word.strip() for word in line.split('|') if word.strip()]
        translated_or_original_words = []

        for word in original_words:
            # поиск перевода в словаре
            # Если не найдено, то ОРИГИНАЛЬНОЕ СЛОВО (word)
            processed_word = translations_dict.get(word.lower(), word)
            translated_or_original_words.append(processed_word)

        # собираем строку обратно с разделителем ' | '
        if translated_or_original_words:
             output_lines.append(" | ".join(translated_or_original_words))

    return "\n\n".join(output_lines) # пустая строка между резами



if __name__ == "__main__":
    input_string = """
zukzikkə | məgxuki | вorş | məsruji
ləmsistan | dlitton | prezident | dkomunisti
ruznamь | gavo | qarxa | dvilə
darəçə | vudun | bkişjutə | bǝşəjuvь
vədta | dkule | kəpitalistetə | dqapital
krьsţjana | mşijxəjə | qəddijşi | mədxərtə
səmt | xzuri | pelxə | tupənq | supraxana
məkupi | paşuţь | ţrapa | cjələ
ruznamь | qənunə | ktivjə | bcirij
bxela | pijşɩ | bijovilə | lkis
xləjbə | snədtə | şəxlipə | industrializatsija
cərcərtə | nvəxə | niqo | kəlвə
вrьjra | miko | nьgaranuta | ptiltə
xьjjal | ţmara | pikkir | вləsə
məqdənə | şotapaja | dməxnəqtə | maqţalta
dakrəj | sovxoze | sənjənə | dkolxoz
hoğə | avtomobijl | qujrəti | kombarь | domna
agrotexnikə | bjulpana | texnikə | tusə
sajjarajь | stakan | рənsil | vərəqə
tuç | miljuni | snetə | bəlpi
"""

    # файл с лексемами и переводами
    lexemes_filename = "lexemes.txt"
    #сюда переводы..
    translations_map = load_translations(lexemes_filename)

    if translations_map:
        result_text = translate_lines_keep_original(input_string, translations_map)
        print("\n--- Результат замены ---")
        print(result_text)
    else:
        print("\nЗамена не выполнена из-за ошибки загрузки переводов.")



Загрузка переводов из файла: lexemes.txt
Загружено переводов для 3902 уникальных форм слов.

--- Результат замены ---
бледный | улыбаться; насмешить | борщ | хмуриться

ləmsistan | dlitton | prezident | dkomunisti

ruznamь | gavo | ворон | dvilə

darəçə | vudun | bkişjutə | bǝşəjuvь

vədta | dkule | kəpitalistetə | dqapital

христианин | христианин | qəddijşi | mədxərtə

метод, навык, умение | xzuri | pelxə | tupənq | столовая

опускать; поклониться | протягивать | бросать | махнуть, пожать (чем-то)

ruznamь | закон, правило | ktivjə | bcirij

bxela | pijşɩ | bijovilə | lkis

xləjbə | опора | şəxlipə | industrializatsija

крик | лаять | niqo | собака

блеклый | miko | nьgaranuta | фитиль

мысль, мнение, намерение | хоронить; погружаться, вязнуть | 1) мечтательность, задумчивость | поразить

жгучий; жалостный | şotapaja | dməxnəqtə | казнь

dakrəj | sovxoze | злобный | dkolxoz

hoğə | avtomobijl | qujrəti | kombarь | domna

agrotexnikə | bjulpana | texnikə | tusə

sajjarajь | stakan | к

In [ ]:
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# --- Имена файлов ---
input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt'
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final.txt'

# --- Вспомогательные функции ---

def similarity(a, b):
    """Схожесть строк в % (без учета регистра)."""
    if not a or not b: return 0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100

def load_prob_final_lemmas(filename):
    """Загружает леммы из prob_final_file в словарь {слово_lower: лемма}."""
    start_time = time.time()
    print(f"Загрузка лемм из '{filename}'...")
    lemma_map = {}
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    lemma = match.group(2).strip()
                    if word and lemma:
                        lemma_map[word.lower()] = lemma # Ключ всегда lowercase
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} лемм из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск в нем будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """Загружает 'Слово:' из photo_problems.txt в список."""
    start_time = time.time()
    print(f"Загрузка проблемных слов из '{filename}'...")
    word_list = []
    pattern = re.compile(r"Слово:\s*([^\s,]+)\s*,")
    loaded_count = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                match = pattern.search(line)
                if match:
                    word = match.group(1)
                    if word:
                        word_list.append(word)
                        loaded_count += 1
        elapsed_time = time.time() - start_time
        print(f"Загружено {loaded_count} слов из '{filename}' за {elapsed_time:.2f} сек.")
    except FileNotFoundError:
        print(f"Предупреждение: Файл '{filename}' не найден. Поиск схожих слов будет пропущен.")
    except Exception as e:
        print(f"Ошибка при чтении файла '{filename}': {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """
    Ищет первое слово в word_list (длина > 3, схожесть >= threshold).
    Приоритет словам со строчной буквы. Сравнение без учета регистра.
    """
    if not target_word: return None
    best_match_lower = None
    best_match_upper = None
    for word in word_list:
        if len(word) > 3:
            sim_score = similarity(target_word, word) # Сравнение уже без регистра
            if sim_score >= threshold:
                # Проверяем регистр кандидата из списка
                if not word or not word[0].isalpha() or word[0].islower():
                    if best_match_lower is None:
                        best_match_lower = word
                        return best_match_lower # Нашли лучший (строчный), выходим
                elif best_match_upper is None:
                     best_match_upper = word # Запоминаем первый прописной
    return best_match_upper # Возвращаем прописной, если строчный не найден

# --- ОБНОВЛЕННАЯ Функция apply_fallback_logic ---
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
     """Ищет лемму в словарях или возвращает токен/слово, учитывая регистр и позицию."""
     if not word: return token or "" # Возвращаем токен или пустую строку, если слова нет

     lemma_to_add = None

     # --- Шаг 2: Поиск в prob_final_lemmas ---
     # Ключ для поиска в словаре ВСЕГДА в нижнем регистре
     search_key_prob = word.lower()
     found_lemma_in_prob = prob_lemmas.get(search_key_prob)

     if found_lemma_in_prob:
         # ПРАВИЛО 2: Если исходное 'Слово:' (word) начиналось с большой буквы, используем его
         if word and word[0].isupper(): # Проверяем регистр исходного слова
             lemma_to_add = word
         else:
             lemma_to_add = found_lemma_in_prob
         # Нашли на шаге 2, выходим
         #print(f"  [DEBUG] Шаг 2: Найдено в prob_final. Итог: '{lemma_to_add}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 3: Поиск схожего в clustered_problem_words ---
     # ПРАВИЛО 1: Определяем цель поиска для similarity
     search_target_problems = word.lower() if is_first_word else word
     found_similar = find_similar_word(search_target_problems, problem_words, 70)

     if found_similar:
         lemma_to_add = found_similar
         # Нашли на шаге 3, выходим
         #print(f"  [DEBUG] Шаг 3: Найдено похожее '{found_similar}' для '{search_target_problems}'. Итог: '{lemma_to_add}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 4: Fallback на Токен ---
     if token:
         lemma_to_add = token
         # Нашли на шаге 4, выходим
         #print(f"  [DEBUG] Шаг 4: Использован токен '{token}' для слова '{word}'")
         return lemma_to_add

     # --- Шаг 5: Самый крайний fallback на исходное Слово ---
     lemma_to_add = word # Если дошли сюда, используем исходное слово
     #print(f"  [DEBUG] Шаг 5: Использовано исходное слово '{word}'")
     return lemma_to_add
# --- Конец ОБНОВЛЕННОЙ функции ---

# --- Основная логика ---

# 1. Проверка файлов и загрузка
if not os.path.exists(input_main_file):
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)

print("--- Загрузка вспомогательных данных ---")
overall_start_time = time.time()
prob_final_lemmas = load_prob_final_lemmas(prob_final_file)
clustered_problem_words = load_clustered_problem_words(problems_clustered_file)
print("--- Загрузка завершена ---")

# 2. Обработка основного файла
results = []
current_sentence_text = None
current_lemmas = []
current_word = None
current_token = None
is_first_word_in_sentence = False # Флаг первого слова
processed_lines_count = 0
processed_sentences_count = 0
last_print_time = time.time()
line_num = 0 # Инициализация перед циклом

print(f"--- Обработка основного файла '{input_main_file}' ---")
try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line_num, line in enumerate(infile, 1):
            processed_lines_count += 1
            line = line.strip()

            current_time = time.time()
            if current_time - last_print_time > 5:
                print(f"Обработано строк: {processed_lines_count}, найдено предложений: {processed_sentences_count}...")
                last_print_time = current_time

            if not line: continue

            # --- Начало нового предложения ---
            if line.startswith("Предложение:"):
                if current_sentence_text is not None:
                    if current_word: # Завершаем последнее слово пред. предложения
                        # Передаем актуальный флаг is_first_word_in_sentence
                        final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                        if final_lemma: current_lemmas.append(final_lemma)
                    if current_lemmas:
                        results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
                        processed_sentences_count += 1

                # Начинаем новое
                try:
                    current_sentence_text = line.split(": ", 1)[1]
                except IndexError:
                    print(f"Предупреждение (строка {line_num}): Не удалось извлечь текст предложения: {line}")
                    current_sentence_text = ""
                current_lemmas = []
                current_word = None
                current_token = None
                is_first_word_in_sentence = True # Устанавливаем флаг

            # --- Новое слово ---
            elif line.startswith("Слово:"):
                 if current_word: # Завершаем предыдущее слово
                     # Передаем актуальный флаг is_first_word_in_sentence
                     final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
                     if final_lemma: current_lemmas.append(final_lemma)
                     # Сбрасываем флаг, если это было первое слово
                     if is_first_word_in_sentence:
                         is_first_word_in_sentence = False

                 # Запоминаем новое слово
                 try:
                     current_word = line.split(": ", 1)[1]
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь слово: {line}")
                     current_word = None
                 current_token = None # Сбрасываем токен

            # --- Токен ---
            elif line.startswith("Токен:") and current_word:
                 try:
                     current_token = line.split(": ", 1)[1]
                     if not current_token: current_token = current_word
                 except IndexError:
                     print(f"Предупреждение (строка {line_num}): Не удалось извлечь токен: {line}")
                     current_token = current_word

            # --- Лемма ---
            elif line.startswith("Лемма:") and current_word:
                lemma_from_file = ""
                try:
                    if ": " in line:
                        lemma_part = line.split(": ", 1)[1]
                        lemma_from_file = lemma_part.split('/', 1)[0].strip()
                    else: lemma_from_file = ""
                except IndexError: lemma_from_file = ""

                # --- Применяем иерархию ---
                lemma_to_add = None
                # Шаг 1: Лемма из файла (если >= 4 символов)
                if lemma_from_file and len(lemma_from_file) >= 4:
                    lemma_to_add = lemma_from_file
                    #print(f"  [DEBUG] Шаг 1: Использована лемма из файла: '{lemma_to_add}' для слова '{current_word}'") # Отладка
                else:
                    # Шаги 2-5 (с учетом флага is_first_word)
                    lemma_to_add = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)

                if lemma_to_add: current_lemmas.append(lemma_to_add)

                # Сбрасываем флаг первого слова, если он еще стоял
                if is_first_word_in_sentence:
                    is_first_word_in_sentence = False

                # Сбрасываем состояние слова
                current_word = None
                current_token = None

    # --- Обработка последнего предложения ---
    if current_sentence_text is not None:
        if current_word: # Завершаем самое последнее слово
            final_lemma = apply_fallback_logic(current_word, current_token, prob_final_lemmas, clustered_problem_words, is_first_word_in_sentence)
            if final_lemma: current_lemmas.append(final_lemma)
        if current_lemmas:
             results.append({"sentence": current_sentence_text, "lemmas": current_lemmas})
             processed_sentences_count += 1

except FileNotFoundError:
    print(f"Критическая ошибка: Основной входной файл '{input_main_file}' не найден.")
    exit(1)
except Exception as e:
    print(f"Критическая ошибка при обработке основного файла (около строки {line_num}): {e}")
    traceback.print_exc()
    exit(1)

processing_time = time.time() - overall_start_time
print(f"--- Обработка основного файла завершена за {processing_time:.2f} сек. Обработано строк: {processed_lines_count} ---")

# 4. Запись результатов
print(f"--- Запись результатов в '{output_final_sentences_file}' ---")
write_start_time = time.time()
if results:
    try:
        with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
            for item in results:
                lemma_string = ' '.join(map(str, item['lemmas']))
                outfile.write(lemma_string.strip() + '\n')
        write_time = time.time() - write_start_time
        print(f"Запись завершена за {write_time:.2f} сек. Сохранено предложений: {len(results)}")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_final_sentences_file}': {e}")
else:
    print("Нет данных для записи в выходной файл.")

total_time = time.time() - overall_start_time
print(f"--- Скрипт завершил работу за {total_time:.2f} сек. ---")


In [4]:
!pip list

Package                   Version
------------------------- --------------
anyio                     4.7.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     24.3.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.11
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.21.1
fonttools                 4.56.0
fqdn                      1.5.1
fuzzywuzzy                0.18.0
gensim                    4.3.3
h11                       0.14.0
httpcore                  1.0.7
httpx       

In [5]:
import pkg_resources

installed_packages = pkg_resources.working_set
installed_packages_list = sorted([f"{p.key}=={p.version}" for p in installed_packages])
for pkg in installed_packages_list:
    print(pkg)


anyio==4.7.0
argon2-cffi-bindings==21.2.0
argon2-cffi==23.1.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.4
attrs==24.3.0
autocommand==2.2.2
babel==2.16.0
backports.tarfile==1.2.0
beautifulsoup4==4.12.3
bleach==6.2.0
certifi==2024.12.14
cffi==1.17.1
charset-normalizer==3.4.1
click==8.1.8
comm==0.2.2
contourpy==1.3.1
cycler==0.12.1
debugpy==1.8.11
decorator==5.1.1
defusedxml==0.7.1
executing==2.1.0
fastjsonschema==2.21.1
fonttools==4.56.0
fqdn==1.5.1
fuzzywuzzy==0.18.0
gensim==4.3.3
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
idna==3.10
importlib-metadata==8.0.0
importlib-resources==6.4.5
inflect==7.3.1
ipykernel==6.29.5
ipython==8.31.0
ipywidgets==8.1.5
isoduration==20.11.0
jaraco.collections==5.1.0
jaraco.context==5.3.0
jaraco.functools==4.0.1
jaraco.text==3.12.1
jedi==0.19.2
jinja2==3.1.5
joblib==1.4.2
json5==0.10.0
jsonpointer==3.0.0
jsonschema-specifications==2024.10.1
jsonschema==4.23.0
jupyter-client==8.6.3
jupyter-console==6.6.3
jupyter-core==5.7.2
jupyter-events==0.11.0
jupyter-

/tmp/ipykernel_8750/509726288.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
